# Configure KBase Jupyter Dev Environment

In [1]:
%run sludgecommutil.py
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
# %tb

['/Users/andrewfreiburger/Documents/Research/sludge',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python39.zip',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/lib-dynload',
 '',
 '/Users/andrewfreiburger/Library/Python/3.9/lib/python/site-packages',
 '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/site-packages',
 '/Users/andrewfreiburger/Documents/Research']

python version 3.9.6
KBBaseModules 0.0.1


/Users/andrewfreiburger/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


modelseedpy 0.3.3
cobrakbase 0.4.0
Output files printed to:/Users/andrewfreiburger/Documents/Research/sludge/sludge/nboutput when using KBDevUtils.output_dir


# Processing ASV mappings: new code

In [2]:
original_mappings = util.load("Test_AGORA2_Bioreactor_over95_best_mappings_02-14-25")
asv_mappings = {}
genomes_mapped = {}
for asv in original_mappings.keys():
    for genome_id in original_mappings[asv][1]:
        if asv not in asv_mappings:
            asv_mappings[asv] = []
        if genome_id not in asv_mappings[asv]:
            asv_mappings[asv].append(genome_id)
            if genome_id not in genomes_mapped:
                genomes_mapped[genome_id] = []
            genomes_mapped[genome_id].append(asv)
util.save("asv_mappings", asv_mappings)
util.save("genomes_mapped", genomes_mapped)
asv_list = list(asv_mappings.keys())
asvset_asvs = {}
asv_asvset = {}
asvset_genomes = {}
genome_asvsets = {}
for i,asv in enumerate(asv_list):
    if asv not in asv_asvset:
        asv_asvset[asv] = asv
        asvset_asvs[asv] = [asv]
        asvset_genomes[asv] = asv_mappings[asv].copy()
        for j in range(i+1,len(asv_list)):
            if asv_list[j] not in asv_asvset:
                common = set(asv_mappings[asv]) & set(asv_mappings[asv_list[j]])
                score = len(common)/min(len(asv_mappings[asv]),len(asv_mappings[asv_list[j]]))
                if score >= 0.5:
                    asv_asvset[asv_list[j]] = asv
                    asvset_asvs[asv].append(asv_list[j])
                    for item in asv_mappings[asv_list[j]]:
                        if item not in asvset_genomes[asv]:
                            asvset_genomes[asv].append(item)          
util.save("asvset_asvs", asvset_asvs)
util.save("asv_asvset", asv_asvset)
util.save("asvset_genomes", asvset_genomes)
util.save("genome_asvsets", genome_asvsets)
print("Genomes:",len(genomes_mapped))
print("ASVs:",len(asv_mappings))
print("ASV sets:",len(asvset_asvs))

Genomes: 1038
ASVs: 117
ASV sets: 63


# Checking if all mapped genomes are in AGORA workspace

In [3]:
ws_client = util.ws_client()
output = ws_client.list_objects({"ids":[178418]})
genome_hash = {}
for item in output:
    genome_hash[item[1]] = item
asvssets_with_missing_genomes = {}
missing_genomes = {}
asvset_genomes = util.load("asvset_genomes")
for asvset in asvset_genomes:
    for genomeid in asvset_genomes[asvset]:
        if genomeid not in genome_hash and genomeid+".RAST" not in genome_hash:
            if genomeid not in missing_genomes:
                missing_genomes[genomeid] = []
            if asvset not in missing_genomes[genomeid]:
                missing_genomes[genomeid].append(asvset)
            if asvset not in asvssets_with_missing_genomes:
                asvssets_with_missing_genomes[asvset] = []
            if genomeid not in asvssets_with_missing_genomes[asvset]:
                asvssets_with_missing_genomes[asvset].append(genomeid)
util.save("missing_genomes", missing_genomes)
util.save("asvssets_with_missing_genomes", asvssets_with_missing_genomes)
for genomeid in missing_genomes:
    print(genomeid,len(missing_genomes[genomeid]))
for asvset in asvssets_with_missing_genomes:
    print(asvset,len(asvssets_with_missing_genomes[asvset]),len(asvset_genomes[asvset]))
    if len(asvset_genomes[asvset]) <= 2:
        for genomeid in asvset_genomes[asvset]:
            if genomeid in missing_genomes:
                print(genomeid)

GCA_000210455.1 1
GCF_000213675.1 1
GCF_000234645.2 1
GCF_000213735.1 1
GCF_000213755.1 1
GCF_000213455.1 1
GCF_001471365.2 1
GCF_008247765.1 1
GCF_000268165.1 1
GCF_000322785.1 1
GCF_000427055.1 1
GCF_006740045.1 1
b8be4a864e591a2801a0b9194efc1060 1 20
ac164649e33237f799c1ffee4f344d5d 8 608
27acca0f07269de77bf31e1b93ecd63a 2 131
efde527a9af4cac96884dbd692f6e11c 1 13


# Developing names for ASV sets

In [4]:
ws_client = util.ws_client()
output = ws_client.list_objects({
    "ids":[178418],
    "includeMetadata":1,
    "type":"KBaseGenomes.Genome"
})
asvset_genomes = util.load("asvset_genomes")
asvname_taxonomy = {}
name_taxonomy = {}
genome_hash = {}
for item in output:
    genome_hash[item[1]] = item
util.save("genome_hash",genome_hash)
asvset_genres = {}
genre_genomes = {}
for asvset in asvset_genomes:
    asvset_genres[asvset] = {}
    for genome in asvset_genomes[asvset]:
        if genome in genome_hash:
            name = genome_hash[genome][10]["Name"]
            genre = name.split(" ")[0]
            if genre[0] == "[":
                genre = genre[1:-1]
            if genre not in asvset_genres[asvset]:
                genre_genomes[genre] = genome_hash[genome]
                asvset_genres[asvset][genre] = 0
            name_taxonomy[genre] = genome_hash[genome][10]["Taxonomy"]
            asvset_genres[asvset][genre] += 1
        elif genome+".RAST" in genome_hash:
            name = genome_hash[genome+".RAST"][10]["Name"]
            genre = name.split(" ")[0]
            if genre[0] == "[":
                genre = genre[1:-1]
            if genre not in asvset_genres[asvset]:
                genre_genomes[genre] = genome_hash[genome+".RAST"]
                asvset_genres[asvset][genre] = 0
            name_taxonomy[genre] = genome_hash[genome+".RAST"][10]["Taxonomy"]
            asvset_genres[asvset][genre] += 1
util.save("asvset_genres",asvset_genres)
util.save("genre_genomes",genre_genomes)
asvset_names = {}
genre_counts = {}
for asvset in asvset_genres:
    best = None
    best_score = 0
    for genre in asvset_genres[asvset]:
        if asvset_genres[asvset][genre] > best_score:
            best = genre
            best_score = asvset_genres[asvset][genre]
    if not best:
        best = "none"
    if best not in genre_counts:
        genre_counts[best] = 0
    genre_counts[best] += 1
    asvset_names[asvset] = best+"."+str(genre_counts[best])
    asvname_taxonomy[asvset_names[asvset]] = name_taxonomy[best]
util.save("genre_counts",genre_counts)
util.save("asvset_names",asvset_names)
util.save("asvname_taxonomy",asvname_taxonomy)

# Pulling genomes and getting functions hash from all genomes

In [7]:
#Now building asv genome objects and saving to file (will be loaded to KBase)
from cobrakbase.core.kbasegenome.genome import KBaseGenome
asvset_asvs = util.load("asvset_asvs")
asvset_genomes = util.load("asvset_genomes")
asvset_names = util.load("asvset_names")
asvset_names = util.load("asvset_names")
genre_genomes = util.load("genre_genomes")
genome_hash = util.load("genome_hash")
functions = util.load("functions")
feature_aliases = util.load("feature_aliases")
feature_probabilities = util.load("feature_probabilities")
features = {}
workspace = 207617
done = []
for asvset in asvset_genomes:
    asvname = asvset_names[asvset]
    if asvname not in done:
        genre = asvname.split(".")[0]
        print("Processing ",asvname)
        #Loading template genome
        template_genome = util.load("TemplateGenome")
        #Setting metadata
        template_genome["assembly_ref"] = str(workspace)+"/"+asvname+".assembly"
        template_genome["dna_size"] = 0
        template_genome["scientific_name"] = genre_genomes[genre][10]["Name"]
        template_genome["taxonomy"] = genre_genomes[genre][10]["Taxonomy"]
        template_genome["gc_content"] = genre_genomes[genre][10]["GC content"]
        template_genome["id"] = asvname
        template_genome["contig_ids"] = []
        template_genome["contig_lengths"] = []
        template_genome["num_contigs"] = 0   
        template_genome["source"] = "WesternDiet"
        template_genome["warnings"] = ["Artificial genome created to support ASV modeling"]
        template_genome["features"] = []
        template_genome["cdss"] = []
        template_genome["source_id"] = "|".join(asvset_asvs[asvset])
        template_genome["molecule_type"] = "ASV derived genome"
        template_genome["genome_type"] = "ASV derived genome"
        template_genome["domain"] = genre_genomes[genre][10]["Domain"]
        template_genome["ontologies_present"] = {}
        template_genome["ontology_events"] = []
        md5_list = []
        if asvname not in functions:
            functions[asvname] = {}
        for genome_id in asvset_genomes[asvset]:
            if genome_id+".RAST" in genome_hash:
                info = genome_hash[genome_id+".RAST"]
                genome = util.msrecon.kbase_api.get_object(genome_id+".RAST",178418)
                genome_functions = {}
                for ftr in genome["features"]:
                    if "functions" in ftr:
                        for function in ftr["functions"]:
                            if function not in functions[asvname]:
                                ftrid = asvname+"_"+str(len(template_genome["contig_ids"])+1)
                                template_genome["contig_ids"].append(ftrid+".contig")
                                template_genome["contig_lengths"].append(len(ftr["dna_sequence"]))
                                template_genome["num_contigs"] += 1
                                template_genome["dna_size"] += len(ftr["dna_sequence"])
                                result = hashlib.md5(ftr["protein_translation"].encode())
                                md5 = result.hexdigest()
                                result = hashlib.md5(ftr["dna_sequence"].encode())
                                dnamd5 = result.hexdigest()
                                md5_list.append(md5)
                                functions[asvname][function] = {"ftrid":ftrid, "probability":1}
                                features[ftrid] = {
                                    "aliases": [],
                                    "cdss": [
                                        ftrid+".CDS"
                                    ],
                                    "functions":[function],
                                    "dna_sequence": ftr["dna_sequence"],
                                    "dna_sequence_length": len(ftr["dna_sequence"]),
                                    "id": ftrid,
                                    "location": [
                                        [
                                            ftrid+".contig",
                                            1,
                                            "+",
                                            len(ftr["dna_sequence"])
                                        ]
                                    ],
                                    "md5": dnamd5,
                                    "ontology_terms": {},
                                    "protein_md5": md5,
                                    "protein_translation": ftr["protein_translation"],
                                    "protein_translation_length": len(ftr["protein_translation"]),
                                    "warnings": []
                                }
                                cdsftr = features[ftrid].copy()
                                del cdsftr["cdss"]
                                cdsftr["id"] = ftrid+".CDS"
                                cdsftr["parent_gene"] = ftrid
                                if "aliases" in ftr and len(ftr["aliases"]) >= 1:
                                    features[ftrid]["aliases"].append(["gene",str(info[6])+"/"+str(info[0])+"."+ftr["aliases"][0][1]])
                                    if ftrid not in feature_aliases:
                                        feature_aliases[ftrid] = []
                                    feature_aliases[ftrid].append([ftr["aliases"][0][1]])
                                template_genome["features"].append(features[ftrid])
                                template_genome["cdss"].append(cdsftr)
                            elif function not in genome_functions:#Don't want to count same function twice in a genome
                                ftrid = functions[asvname][function]["ftrid"]
                                functions[asvname][function]["probability"] += 1
                                if "aliases" in ftr and len(ftr["aliases"]) >= 1:
                                    features[ftrid]["aliases"].append(["gene",str(info[6])+"/"+str(info[0])+"."+ftr["aliases"][0][1]])
                                    if ftrid not in feature_aliases:
                                        feature_aliases[ftrid] = []
                                    feature_aliases[ftrid].append(ftr["aliases"][0][1])
                            genome_functions[function] = True
        for func in functions[asvname]:
            functions[asvname][func]["probability"] = functions[asvname][func]["probability"]/len(asvset_genomes[asvset])
            feature_probabilities[functions[asvname][func]["ftrid"]] = functions[asvname][func]["probability"]
        template_genome["feature_counts"] = {
            "CDS":len(template_genome["features"]),
            "gene":len(template_genome["features"]),
            "non_coding_features":0,
            "protein_encoding_gene":len(template_genome["features"])
        }                    
        md5_list.sort()
        result = hashlib.md5(";".join(md5_list).encode())
        template_genome["md5"] = result.hexdigest()
        #Writing FASTA
        ofile = open("Assemblies/"+asvname+".fasta", "w")
        for func in functions[asvname]:
            ofile.write(">" + functions[asvname][func]["ftrid"] + "\n" +features[functions[asvname][func]["ftrid"]]["dna_sequence"] + "\n")
        ofile.close()
        #Saving genome
        util.save(asvname, template_genome)
        util.save("feature_aliases", feature_aliases)
        util.save("functions", functions)
        util.save("feature_probabilities", feature_probabilities)     

1740027402.870403 ERROR: Requested data feature_probabilities doesn't exist at /Users/andrewfreiburger/Documents/Research/sludge/sludge/../data/feature_probabilities.json


ValueError: Requested data feature_probabilities doesn't exist at /Users/andrewfreiburger/Documents/Research/sludge/sludge/../data/feature_probabilities.json

# Loading assemblies to KBase

In [5]:
#Loading clade genome assemblies to KBase using SDK callbacks
import shutil
util.sdk_dir_perms()
afu = util.afu_client()
asvset_genomes = util.load("asvset_genomes")
asvset_names = util.load("asvset_names")
for asvset in asvset_genomes:
    asvname = asvset_names[asvset]
    shutil.copy2("Assemblies/"+asvname+".fasta", '/scratch/shared/sdkmount/kb_sdk_home/run_local/workdir/tmp/'+asvname+".fasta")
    params = {
        'file': {
            'path': '/kb/module/work/tmp/'+asvname+".fasta"
        },
        'workspace_id': 181152,
        'assembly_name': asvname+".assembly"
    }
    result = afu.save_assembly_from_fasta2(params)

# Saving genomes to KBase

In [5]:
#Loading clade genomes to KBase using annotation API, which contains an SDK callback
feature_probabilities = util.load("feature_probabilities")
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
workspace = 181152
anno = util.kbdevutil.anno_client()
anno.clients["GenomeFileUtil"] = util.kbdevutil.gfu_client()
asvset_genomes = util.load("asvset_genomes")
asvset_names = util.load("asvset_names")
done = []#["Lactobacillus.3","Clostridium.11","Muribaculum.1"]#This genome actually failed...
finished = util.load("finished_genomes")
for asvset in asvset_genomes:
    asvname = asvset_names[asvset]
    if asvname not in done and asvname not in finished:
        genome = util.kbdevutil.load(asvname)
        genome["assembly_ref"] = str(workspace)+"/"+asvname+".assembly"
        genome["source_id"] = asvname
        events = [{
            "id": "SSO",
            "event_id":"RAST:SSO:"+str(timestamp),
            "description": "RAST annotation of clade genome",
            "ontology_id": "SSO",
            "method": "RAST",
            "method_version": "1.9.5",
            "timestamp": str(timestamp),
            "ontology_terms": {},
        }]
        for ftr in genome["features"]:
            events[0]["ontology_terms"][ftr["id"]] = [{"term":"SSO:"+ftr["functions"][0],"evidence":{"scores":{"pgprob":feature_probabilities[ftr["id"]]}}}]
        output = anno.add_annotation_ontology_events({
            "output_workspace":workspace,
            "events":events,
            "overwrite_matching":True,
            "object":genome,
            "type":"KBaseGenomes.Genome",
            "output_name":asvname+".ASV",
            "save":1
        })
        finished.append(asvname)
        util.save("finished_genomes",finished)

# Construct the ASV models, gapfilling in auxotrophy and anaerobic glucose minimal media - code cell from KBase copied here for reference

In [ ]:
import requests
import os
import io
import sys
from biokbase.workspace.client import Workspace
from biokbase.narrative.jobs.appmanager import AppManager
current_ws = os.environ['KB_WORKSPACE_ID']

ws_client = Workspace()
genomes = ws_client.list_objects({"ids":[181152],"type":"KBaseGenomes.Genome"})
genomes = genomes[2:]
for genome in genomes:
    AppManager().run_app(
        "ModelSEEDReconstruction/build_metabolic_models",
        {
            "genome_refs": ["181152/"+genome[1]],
            "suffix": ".mdl",
            "run_gapfilling": 1,
            "gapfilling_media_list": ["181152/AuxoMedia","181152/AnaerobicGMM"],
            "change_to_complete": 0,
            "atp_safe": 1,
            "forced_atp_list": [],
            "gs_template": "auto",
            "gs_template_ref": None,
            "core_template_ref": None,
            "gapfilling_mode": "Sequential",
            "expression_refs": [],
            "expression_conditions": []
        },
        tag="release",
        version="2.0.0",
        cell_id="c4caea9f-c131-4b17-88e3-19e2b54b655a",
        run_id="9e3a761b-3ca7-4b5a-9197-ae4c858edef2"
    )

/opt/env/python3_modelseed/lib/python3.11/site-packages/urllib3/util/ssl_.py:262: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  context = SSLContext(ssl_version or ssl.PROTOCOL_SSLv23)
/opt/env/python3_modelseed/lib/python3.11/site-packages/urllib3/connection.py:374: DeprecationWarning: ssl.match_hostname() is deprecated
  match_hostname(cert, asserted_hostname)
/opt/env/python3_modelseed/lib/python3.11/site-packages/urllib3/util/ssl_.py:262: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  context = SSLContext(ssl_version or ssl.PROTOCOL_SSLv23)
/opt/env/python3_modelseed/lib/python3.11/site-packages/urllib3/connection.py:374: DeprecationWarning: ssl.match_hostname() is deprecated
  match_hostname(cert, asserted_hostname)


N


/opt/env/python3_modelseed/lib/python3.11/site-packages/urllib3/util/ssl_.py:262: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  context = SSLContext(ssl_version or ssl.PROTOCOL_SSLv23)
/opt/env/python3_modelseed/lib/python3.11/site-packages/urllib3/connection.py:374: DeprecationWarning: ssl.match_hostname() is deprecated
  match_hostname(cert, asserted_hostname)
1710772440.7858362 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1710772547.7780695 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1710772636.7279124 INFO: Compartment `e0` sounds like an external compartment. Using this one without counting boundary reactions.
1710772636.835424 INFO: Compartment `e0` sounds like an external compartment. Using this one without counting boundary reactions.
1710772636.9265068 INFO: Compartment `e0` sounds like an external compartment. Using this one without counting boundary reactions.
1710772637.0251577

Removing ungapfillable media Etho


1710772796.8349624 INFO: Objective with gapfill database:0.0; min objective:0.01
1710772797.1610184 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1710772903.9313705 WARNING: No gapfilling solution found before filtering for mal-L activating rxn00062_c0
1710772904.067492 INFO: Objective with gapfill database:1.0000000000000036; min objective:0.01


...

Adding gapfilling {'growth': 0, 'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f3df1cc6d10>, 'target': 'rxn00062_c0', 'minobjective': 0.01, 'binary_check': False, 'new': {'rxn24608_c0': '>', 'rxn24609_c0': '>', 'rxn24610_c0': '>', 'rxn24613_c0': '>', 'rxn09318_c0': '<', 'rxn15961_c0': '>', 'rxn24611_c0': '>'}, 'reversed': {}}
{<modelseedpy.core.msmedia.MSMedia object at 0x7f3df1ad6c90>: {'growth': 0, 'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f3df1ad6c90>, 'target': 'rxn00062_c0', 'minobjective': 2, 'binary_check': False, 'new': {'rxn09269_c0': '>'}, 'reversed': {}}, <modelseedpy.core.msmedia.MSMedia object at 0x7f3df1a8b5d0>: {'growth': 0, 'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f3df1a8b5d0>, 'target': 'rxn00062_c0', 'minobjective': 0.01, 'binary_check': False, 'new': {'rxn03020_c0': '>', 'rxn15961_c0': '>'}, 'reversed': {}}, <modelseedpy.core.msmedia.MSMedia object at 0x7f3df1a89190>: {'growth': 0, 'media': <modelseedpy.core.msmedia.MSMedia objec

1710773277.8779063 INFO: Setting FBAModel computed_attributes to mdlutl attributes


Failed:<rxn10157_c0
Failed:<rxn00146_c0
Failed:>rxn10123_c0
Failed:>rxn10124_c0
Failed:>rxn00532_c0
Failed:<rxn10155_c0
Failed:>rxn12575_c0
Failed:<rxn00429_c0
Failed:<rxn00145_c0
Failed:>rxn12822_c0
Failed:>rxn00704_c0
Failed:>rxn01671_c0
Failed:<rxn00377_c0
Failed:>rxn00778_c0
Failed:>rxn05206_c0
Failed:<rxn05611_c0
Failed:<rxn14159_c0
Failed:<rxn00154_c0
Failed:>rxn45223_c0
Failed:<rxn48579_c0
Failed:>rxn39679_c0
Failed:<rxn08735_c0
Failed:<rxn08736_c0
Failed:<rxn39860_c0
Failed:<rxn44043_c0
Failed:<rxn13339_c0
Failed:<rxn13321_c0
Failed:<rxn42976_c0
Failed:<rxn20627_c0
Failed:>rxn09193_c0
Failed:<rxn08234_c0
Failed:<rxn23850_c0
Failed:<rxn40368_c0
Failed:>rxn20583_c0
Failed:<rxn16596_c0
Failed:>rxn11667_c0
Failed:<rxn09388_c0
Failed:>rxn16149_c0
Failed:<rxn43657_c0


1710773286.4691327 INFO: Expansion time:empty:8.415746569051407
1710773286.4698327 INFO: Filtered count:39 out of 3345
/scratch/shared/code/ModelSEEDpy/modelseedpy/core/msmodelutl.py:29: DeprecationWarning: invalid escape sequence '\d'
  if re.search("^(cpd\d+)", metabolite.id):
/scratch/shared/code/ModelSEEDpy/modelseedpy/core/msmodelutl.py:30: DeprecationWarning: invalid escape sequence '\d'
  m = re.search("^(cpd\d+)", metabolite.id)
/scratch/shared/code/ModelSEEDpy/modelseedpy/core/msmodelutl.py:35: DeprecationWarning: invalid escape sequence '\d'
  if re.search("^(cpd\d+)", item):
/scratch/shared/code/ModelSEEDpy/modelseedpy/core/msmodelutl.py:36: DeprecationWarning: invalid escape sequence '\d'
  m = re.search("^(cpd\d+)", item)
/scratch/shared/code/ModelSEEDpy/modelseedpy/core/msmodelutl.py:38: DeprecationWarning: invalid escape sequence '\d'
  elif re.search("^(cpd\d+)", metabolite.annotation[anno]):
/scratch/shared/code/ModelSEEDpy/modelseedpy/core/msmodelutl.py:39: Deprecatio

KeyboardInterrupt: 

# Adding probabilities to ASV models

In [24]:
feature_probabilities = util.load("feature_probabilities")
asvset_names = util.load("asvset_names")
asvset_genomes = util.load("asvset_genomes")
probability_finished = []#util.load("probability_finished")
for asvset in asvset_genomes:
    asvname = asvset_names[asvset]
    if asvname not in probability_finished:
        mdlutl = util.msrecon.get_model(asvname+".ASV.mdl",181152)
        for rxn in mdlutl.model.reactions:
                highest_prob = None
                for gene in rxn.genes:
                    if gene.id in feature_probabilities:
                        if highest_prob == None or feature_probabilities[gene.id] > highest_prob:
                            highest_prob = feature_probabilities[gene.id]
                if highest_prob != None:
                    rxn.probability = highest_prob
        #util.msrecon.save_model(mdlutl,181152,asvname+".ASV.mdl")
        probability_finished.append(asvname)
        util.save("probability_finished",probability_finished)

1714110491.7230527 INFO: metabolites 1085
1714110495.821199 INFO: reactions 988
1714110495.9441411 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1714110496.3080332 INFO: Default biomass: [bio1]
1714110496.3407097 INFO: Setting FBAModel computed_attributes to mdlutl attributes
1714110498.8682323 INFO: metabolites 1027
1714110502.4806764 INFO: reactions 988
1714110502.5843725 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1714110502.9308474 INFO: Default biomass: [bio1]
1714110502.9558785 INFO: Setting FBAModel computed_attributes to mdlutl attributes
1714110504.6506052 INFO: metabolites 909
1714110508.013077 INFO: reactions 868
1714110508.1104188 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1714110508.419499 INFO: Default biomass: [bio1]
1714110508.4425995 INFO: Setting FBAModel computed_attributes to mdlutl attributes
1714110509.9471219 INFO: metabolites 1

# Loading metabolomics data

In [6]:
metabolite_df = pd.read_csv('data/metabolomics_log2_fold_change.csv')
columns = metabolite_df.columns.tolist()
metabolites = columns[1:]
metabolomics_data = {}
metabolite_names = {}
samples = []
for index, row in metabolite_df.iterrows():
    samples.append(row["day_diet diff"])
    for met in metabolites:
        if met not in metabolomics_data:
            metabolomics_data[met] = {}
        metabolomics_data[met][row["day_diet diff"]] = row[met]
util.save("metabolomics_data",metabolomics_data)
util.save("samples",samples)
util.save("metabolites",metabolites)
modelseed = ModelSEEDBiochem.get()
for met in metabolites:
    cpdobj = modelseed.get_seed_compound(met)
    if cpdobj:
        metabolite_names[met] = cpdobj.name
    else:
        metabolite_names[met] = met
util.save("metabolite_names",metabolite_names)

1717476819.571839 INFO: load: /Users/chenry/code/ModelSEEDDatabase/Biochemistry/reactions.tsv


1717476820.117903 INFO: load: /Users/chenry/code/ModelSEEDDatabase/Biochemistry/compounds.tsv
1717476828.29903 INFO: load: /Users/chenry/code/ModelSEEDDatabase/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
1717476831.772461 INFO: load: /Users/chenry/code/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
1717476831.847784 INFO: load: /Users/chenry/code/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
1717476831.969643 INFO: load: /Users/chenry/code/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


# Interaction testing for metabolites in models

In [4]:
%run agoracommutil.py
metabolites = util.load("metabolites")
asvset_names = util.load("asvset_names")
asvset_genomes = util.load("asvset_genomes")
phenotype_results = util.load("phenotype_results",default={})
auxo_media = util.msrecon.get_media("181152/AuxoMedia")
agmm_base_media = util.msrecon.get_media("181152/BaseAnaerobicMM")
gmm_base_media = util.msrecon.get_media("181152/BaseAerobicMM")
uptake_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=auxo_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="uptake"
)
excretion_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=auxo_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="excretion"
)
agrowth_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=agmm_base_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="growth"
)
growth_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=gmm_base_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="growth"
)
for asvset in asvset_genomes:
    asvname = asvset_names[asvset]
    if asvname not in phenotype_results and asvname != "Thermus.1":
        phenotype_results[asvname] = {}
        mdlutl = util.msrecon.get_model(asvname+".ASV.mdl",181152)
        mdlutl.solver = 'gurobi'
        if "uptake" not in phenotype_results[asvname]:
            phenotype_results[asvname]["uptake"] = {}
            output = uptake_phenoset.simulate_phenotypes(
                mdlutl,
                multiplier=2,
                add_missing_exchanges=True,
                save_fluxes=False,
                gapfill_negatives=False,
                msgapfill=None,
                test_conditions=None,
                ignore_experimental_data=True
            )
            for index, row in output["details"].iterrows():
                phenotype_results[asvname]["uptake"][row["Phenotype"]] = output["data"][row["Phenotype"]]
            util.save("phenotype_results",phenotype_results)
        if "excretion" not in phenotype_results[asvname]:
            phenotype_results[asvname]["excretion"] = {}
            output = excretion_phenoset.simulate_phenotypes(
                mdlutl,
                multiplier=2,
                add_missing_exchanges=True,
                save_fluxes=False,
                gapfill_negatives=False,
                msgapfill=None,
                test_conditions=None,
                ignore_experimental_data=True
            )
            for index, row in output["details"].iterrows():
                phenotype_results[asvname]["excretion"][row["Phenotype"]] = output["data"][row["Phenotype"]]
            util.save("phenotype_results",phenotype_results)
        if "agrowth" not in phenotype_results[asvname]:
            phenotype_results[asvname]["agrowth"] = {}
            output = agrowth_phenoset.simulate_phenotypes(
                mdlutl,
                multiplier=2,
                add_missing_exchanges=True,
                save_fluxes=False,
                gapfill_negatives=False,
                msgapfill=None,
                test_conditions=None,
                ignore_experimental_data=True
            )
            for index, row in output["details"].iterrows():
                phenotype_results[asvname]["agrowth"][row["Phenotype"]] = output["data"][row["Phenotype"]]
            util.save("phenotype_results",phenotype_results)
        if "growth" not in phenotype_results[asvname]:
            phenotype_results[asvname]["growth"] = {}
            output = growth_phenoset.simulate_phenotypes(
                mdlutl,
                multiplier=2,
                add_missing_exchanges=True,
                save_fluxes=False,
                gapfill_negatives=False,
                msgapfill=None,
                test_conditions=None,
                ignore_experimental_data=True
            )
            for index, row in output["details"].iterrows():
                phenotype_results[asvname]["growth"][row["Phenotype"]] = output["data"][row["Phenotype"]]
            util.save("phenotype_results",phenotype_results)

# Printing interaction matrices

In [3]:
%run agoracommutil.py
metabolites = util.load("metabolites")
phenotype_results = util.load("phenotype_results")
asvset_genomes = util.load("asvset_genomes")
asvset_names = util.load("asvset_names")
records = {
    "combined":[{"ASV":"Name"}],
    "uptake":[{"ASV":"Name"}],
    "excretion":[{"ASV":"Name"}],
    "agrowth":[{"ASV":"Name"}],
    "growth":[{"ASV":"Name"}]
}
metabolite_names = util.load("metabolite_names")
for met in metabolites:
    for item in records:
        records[item][0][met] = metabolite_names[met]
for asvset in asvset_genomes:
    asvname = asvset_names[asvset]
    if asvname not in phenotype_results:
        print("No data for ",asvname)
        continue
    new_record = {
        "combined":{"ASV":asvname},
        "uptake":{"ASV":asvname},
        "excretion":{"ASV":asvname},
        "agrowth":{"ASV":asvname},
        "growth":{"ASV":asvname}
    }  
    for metabolite in metabolites:
        value = ""
        if "uptake" in phenotype_results[asvname]:
            if metabolite in phenotype_results[asvname]["uptake"]:
                new_record["uptake"][metabolite] = phenotype_results[asvname]["uptake"][metabolite]["objective_value"]
                if phenotype_results[asvname]["uptake"][metabolite]["class"] == "P":
                    value += "U"
        if "excretion" in phenotype_results[asvname]:
            if metabolite in phenotype_results[asvname]["excretion"]:
                new_record["excretion"][metabolite] = phenotype_results[asvname]["excretion"][metabolite]["objective_value"]
                if phenotype_results[asvname]["excretion"][metabolite]["class"] == "P":
                    value += "E"
        if "agrowth" in phenotype_results[asvname]:
            if metabolite in phenotype_results[asvname]["agrowth"]:
                new_record["agrowth"][metabolite] = phenotype_results[asvname]["agrowth"][metabolite]["objective_value"]
                if phenotype_results[asvname]["agrowth"][metabolite]["class"] == "P":
                    value += "A"
        if "growth" in phenotype_results[asvname]:
            if metabolite in phenotype_results[asvname]["growth"]:
                new_record["growth"][metabolite] = phenotype_results[asvname]["growth"][metabolite]["objective_value"]
                if phenotype_results[asvname]["growth"][metabolite]["class"] == "P":
                    value += "G"
        new_record["combined"][metabolite] = value
    for item in records:
        records[item].append(new_record[item])
#Printing interation matrices
for item in records:
    df = pd.DataFrame.from_records(records[item])
    df.to_csv(util.output_dir+"/ASV"+item+"Interactions.csv",index=False)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/
No data for  Thermus.1


# Loading Sample data

In [16]:
%run agoracommutil.py
asv_asvset = util.load("asv_asvset")
asv_mappings = util.load("asv_mappings")
sample_df = pd.read_csv('data/mean_abund_perDay_perTreatment_100.csv')
columns = sample_df.columns.tolist()
samples = columns[1:]
asv_abundances = {}
asvset_abundances = {}
sample_abundanes = {}
sample_asvset_abundances = {}
for sample in samples:
    sample_asvset_abundances[sample] = {}
    sample_abundanes[sample] = {
        "asv_total":0,
        "asvset_total":0,
        "mapped_asvs_total":0,
        "unmapped_asvs_total":0,
    }
for index, row in sample_df.iterrows():
    asv_abundances[row["asv"]] = {}
    if row["asv"] in asv_asvset and asv_asvset[row["asv"]] not in asvset_abundances:
        asvset_abundances[asv_asvset[row["asv"]]] = {}
    for sample in samples:
        if row["asv"] in asv_asvset:
            if sample not in asvset_abundances[asv_asvset[row["asv"]]]:
                asvset_abundances[asv_asvset[row["asv"]]][sample] = 0
            asvset_abundances[asv_asvset[row["asv"]]][sample] += row[sample]
        asv_abundances[row["asv"]][sample] = row[sample]
for asvset in asvset_abundances:
    for sample in samples:
        if asvset_abundances[asvset][sample] > 0:
            sample_asvset_abundances[sample][asvset] = asvset_abundances[asvset][sample]
        if sample in asvset_abundances[asvset]:
            sample_abundanes[sample]["asvset_total"] += asvset_abundances[asvset][sample]
for asv in asv_abundances:
    for sample in samples:
        sample_abundanes[sample]["asv_total"] += asv_abundances[asv][sample]
        if asv in asv_mappings:
            sample_abundanes[sample]["mapped_asvs_total"] += asv_abundances[asv][sample]
        else:
            sample_abundanes[sample]["unmapped_asvs_total"] += asv_abundances[asv][sample]
util.save("sample_asvset_abundances",sample_asvset_abundances)
util.save("sample_abundanes",sample_abundanes)
util.save("asv_abundances",asv_abundances)
util.save("asvset_abundances",asvset_abundances)
util.save("samples",samples)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Now we will generate the sample interaction matrix

In [20]:
%run agoracommutil.py
asv_asvset = util.load("asv_asvset")
metabolites = util.load("metabolites")
phenotype_results = util.load("phenotype_results")
asvset_genomes = util.load("asvset_genomes")
asvset_names = util.load("asvset_names")
asvset_abundances = util.load("asvset_abundances")
asvset_abundances = util.load("sample_asvset_abundances")
sample_abundanes = util.load("sample_abundanes")
records = {
    "uptake_bin":[{"Sample":"Name"}],
    "excretion_bin":[{"Sample":"Name"}],
    "agrowth_bin":[{"Sample":"Name"}],
    "growth_bin":[{"Sample":"Name"}],
    "uptake":[{"Sample":"Name"}],
    "excretion":[{"Sample":"Name"}],
    "agrowth":[{"Sample":"Name"}],
    "growth":[{"Sample":"Name"}]
}
metabolite_names = util.load("metabolite_names")
for met in metabolites:
    for item in records:
        records[item][0][met] = metabolite_names[met]
for sample in samples:
    new_record = {
        "uptake_bin":{"Sample":sample},
        "excretion_bin":{"Sample":sample},
        "agrowth_bin":{"Sample":sample},
        "growth_bin":{"Sample":sample},
        "uptake":{"Sample":sample},
        "excretion":{"Sample":sample},
        "agrowth":{"Sample":sample},
        "growth":{"Sample":sample}
    }
    for metabolite in metabolites:
        for item in new_record:
            new_record[item][metabolite] = 0
    for asvset in sample_asvset_abundances[sample]:
        asvname = asvset_names[asvset]
        for metabolite in metabolites:
            if "uptake" in phenotype_results[asvname]:
                if metabolite in phenotype_results[asvname]["uptake"]:
                    new_record["uptake"][metabolite] += sample_asvset_abundances[sample][asvset]/sample_abundanes[sample]["asvset_total"]*phenotype_results[asvname]["uptake"][metabolite]["objective_value"]
                    if phenotype_results[asvname]["uptake"][metabolite]["class"] == "P":
                        new_record["uptake_bin"][metabolite] += sample_asvset_abundances[sample][asvset]/sample_abundanes[sample]["asvset_total"]
            if "excretion" in phenotype_results[asvname]:
                if metabolite in phenotype_results[asvname]["excretion"]:
                    new_record["excretion"][metabolite] += sample_asvset_abundances[sample][asvset]/sample_abundanes[sample]["asvset_total"]*phenotype_results[asvname]["excretion"][metabolite]["objective_value"]
                    if phenotype_results[asvname]["excretion"][metabolite]["class"] == "P":
                        new_record["excretion_bin"][metabolite] += sample_asvset_abundances[sample][asvset]/sample_abundanes[sample]["asvset_total"]
            if "agrowth" in phenotype_results[asvname]:
                if metabolite in phenotype_results[asvname]["agrowth"]:
                    new_record["agrowth"][metabolite] += sample_asvset_abundances[sample][asvset]/sample_abundanes[sample]["asvset_total"]*phenotype_results[asvname]["agrowth"][metabolite]["objective_value"]
                    if phenotype_results[asvname]["agrowth"][metabolite]["class"] == "P":
                        new_record["agrowth_bin"][metabolite] += sample_asvset_abundances[sample][asvset]/sample_abundanes[sample]["asvset_total"]
            if "growth" in phenotype_results[asvname]:
                if metabolite in phenotype_results[asvname]["growth"]:
                    new_record["growth"][metabolite] += sample_asvset_abundances[sample][asvset]/sample_abundanes[sample]["asvset_total"]*phenotype_results[asvname]["growth"][metabolite]["objective_value"]
                    if phenotype_results[asvname]["growth"][metabolite]["class"] == "P":
                        new_record["growth_bin"][metabolite] += sample_asvset_abundances[sample][asvset]/sample_abundanes[sample]["asvset_total"]
    for item in records:
        records[item].append(new_record[item])
#Printing interation matrices
for item in records:
    df = pd.DataFrame.from_records(records[item])
    df.to_csv(util.output_dir+"/Sample"+item+"Interactions.csv",index=False)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Gapfilling phenotype evaluation

In [2]:
%run agoracommutil.py
procid = 0
metabolites = util.load("metabolites")
feature_probabilities = util.load("feature_probabilities")
asvset_genomes = util.load("asvset_genomes")
asvset_names = util.load("asvset_names")
gf_phenotype_results = util.load("new_gf_phenotype_results_"+str(procid),{})
probability_finished = util.load("probability_finished_"+str(procid),[])
problemlist = util.load("problemlist")

auxo_media = util.msrecon.get_media("181152/AuxoMedia")
agmm_base_media = util.msrecon.get_media("181152/BaseAnaerobicMM")
gmm_base_media = util.msrecon.get_media("181152/BaseAerobicMM")
uptake_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=auxo_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="uptake"
)
excretion_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=auxo_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="excretion"
)
growth_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=gmm_base_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="growth"
)
phenosets = {"uptake":uptake_phenoset}#,"excretion":excretion_phenoset,"growth":growth_phenoset}

asvset_list = problemlist
for i,asvname in enumerate(asvset_list):
#asvset_list = list(asvset_genomes.keys())
#for i,asvset in enumerate(asvset_list):
    if i%8 == procid:
        #asvname = asvset_names[asvset]
        if asvname not in probability_finished and asvname != "Thermus.1":
            mdlutl = util.msrecon.get_model(asvname+".ASV.mdl",181152)

            reaction_probabilities = {}
            for rxn in mdlutl.model.reactions:
                highest_prob = None
                for gene in rxn.genes:
                    if gene.id in feature_probabilities:
                        if highest_prob == None or feature_probabilities[gene.id] > highest_prob:
                            highest_prob = feature_probabilities[gene.id]
                if highest_prob != None:
                    rxn.probability = highest_prob
                    reaction_probabilities[rxn.id] = highest_prob

            filters = mdlutl.get_attributes("gf_filter")
            tests = mdlutl.get_atp_tests(core_template=util.msrecon.core_template,atp_media_filename=util.msrecon.module_dir+"/data/atp_medias.tsv",recompute=False)
            msgapfill = MSGapfill(
                mdlutl,
                [util.msrecon.get_template(mdlutl.model.template_ref)],
                [],
                tests,
                blacklist=[],
                default_target="bio1",
                minimum_obj=0.01,
                base_media=None,
                base_media_target_element=None
            )

            #Adding missing transporter for metabolites to gapfilling database
            for cpd in metabolites:
                if "EX_"+cpd+"_e0" not in msgapfill.gfmodelutl.model.reactions:
                    transport = msgapfill.gfmodelutl.add_transport_and_exchange_for_metabolite(cpd,direction="=",prefix="trans",override=False)

            coefficients = {}
            gf_penalties = msgapfill.gfpkgmgr.getpkg("GapfillingPkg").gapfilling_penalties
            gfrxn = 0
            probrxn = 0
            otherrxn = 0
            for reaction in msgapfill.gfmodelutl.model.reactions:
                if reaction.id in gf_penalties:
                    if "forward" in gf_penalties[reaction.id]:
                        gfrxn += 1
                        coefficients[">"+reaction.id] = 1+gf_penalties[reaction.id]["forward"]
                    elif reaction.id in reaction_probabilities:
                        probrxn += 1
                        coefficients[">"+reaction.id] = 1-reaction_probabilities[reaction.id]
                    else:
                        otherrxn += 1
                        coefficients[">"+reaction.id] = 0.95
                    if "reverse" in gf_penalties[reaction.id]:
                        gfrxn += 1
                        coefficients["<"+reaction.id] = 1+gf_penalties[reaction.id]["reverse"]
                    elif reaction.id in reaction_probabilities:
                        probrxn += 1
                        coefficients["<"+reaction.id] = 1-reaction_probabilities[reaction.id]
                    else:
                        otherrxn += 1
                        coefficients["<"+reaction.id] = 0.95
                elif reaction.id in reaction_probabilities:
                    probrxn += 2
                    coefficients[">"+reaction.id] = 1-reaction_probabilities[reaction.id]
                    coefficients["<"+reaction.id] = 1-reaction_probabilities[reaction.id]
                else:
                    otherrxn += 2
                    coefficients[">"+reaction.id] = 0.95
                    coefficients["<"+reaction.id] = 0.95
            print(asvname,"GF:",gfrxn,"Prob:",probrxn,"Other:",otherrxn)

            msgapfill.prefilter(test_conditions=tests,growth_conditions=[],use_prior_filtering=True,base_filter_only=True)
            
            gf_phenotype_results[asvname] = {}
            for phenoid in phenosets:
                gf_phenotype_results[asvname][phenoid] = {}
                output = phenosets[phenoid].simulate_phenotypes(
                    msgapfill.gfmodelutl,
                    multiplier=2,
                    add_missing_exchanges=True,
                    save_fluxes=False,
                    save_reaction_list=True,
                    gapfill_negatives=False,
                    msgapfill=None,
                    test_conditions=None,
                    ignore_experimental_data=True,
                    flux_coefficients=coefficients
                )
                for index, row in output["details"].iterrows():
                    if "reactions" in output["data"][row["Phenotype"]]:
                        output["data"][row["Phenotype"]]["average_probability"] = 0
                        for rxn in output["data"][row["Phenotype"]]["reactions"]:
                            direction = rxn[0:1]
                            rxnid = rxn[1:]
                            if direction == ">":
                                if rxnid not in gf_penalties or "forward" not in gf_penalties[rxnid]:
                                    if rxnid in reaction_probabilities:
                                        output["data"][row["Phenotype"]]["average_probability"] += reaction_probabilities[rxnid]
                                    else:
                                        output["data"][row["Phenotype"]]["average_probability"] += 0.05
                            elif direction == "<":
                                if rxnid not in gf_penalties or "reverse" not in gf_penalties[rxnid]:
                                    if rxnid in reaction_probabilities:
                                        output["data"][row["Phenotype"]]["average_probability"] += reaction_probabilities[rxnid]
                                    else:
                                        output["data"][row["Phenotype"]]["average_probability"] += 0.05
                        output["data"][row["Phenotype"]]["average_probability"] = output["data"][row["Phenotype"]]["average_probability"]/len(output["data"][row["Phenotype"]]["reactions"])
                    gf_phenotype_results[asvname][phenoid][row["Phenotype"]] = output["data"][row["Phenotype"]]
            probability_finished.append(asvname)
            util.save("new_gf_phenotype_results_"+str(procid),gf_phenotype_results)
            util.save("probability_finished_"+str(procid),probability_finished)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1717516004.8265018 INFO: metabolites 1165
1717516005.0642471 INFO: reactions 1070
1717516005.8270679 INFO: Default biomass: [bio1]
1717516005.9175858 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.
1717516006.443178 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.


Getting tests from attributes


1717516016.794209 INFO: cpd35610 not found in model!
1717516016.7950861 CRITICAL: Metabolite cpd35610 not found in model
1717516016.816258 INFO: cpd07053 not found in model!
1717516016.8171258 CRITICAL: Metabolite cpd07053 not found in model
1717516016.8176901 INFO: cpd04945 not found in model!
1717516016.818371 CRITICAL: Metabolite cpd04945 not found in model


Bilophila.1 GF: 13187 Prob: 1722 Other: 3395


...

1717523702.93089 INFO: Objective with gapfill database:0.0; min objective:0.01
/Users/chenry/opt/anaconda3/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
1717523708.7628028 INFO: Expansion time:Ac.NO3:0.826883156999429
1717523708.763595 INFO: Filtered count:0 out of 1704
1717523709.523921 INFO: Expansion time:Ac.O2:0.7598687469999277
1717523709.524446 INFO: Filtered count:0 out of 1704
1717523710.413308 INFO: Expansion time:Etho.O2:0.8881068300006518
1717523710.413882 INFO: Filtered count:0 out of 1704
1717523711.3909812 INFO: Expansion time:For.NO2:0.9763185450001401
1717523711.3916001 INFO: Filtered count:0 out of 1704
1717523712.318198 INFO: Expansion time:For.NO3:0.9256380749993696
1717523712.318748 INFO: Filtered count:0 out of 1704
1717523713.196631 INFO: Expansion time:For.O2:0.8774073059994407
1717523713.197152 INFO: Filtered count:0 out of 1704
1717523714.1637928 INFO: Expan

Getting tests from attributes


1717524070.8650272 INFO: cpd35610 not found in model!
1717524070.8657029 CRITICAL: Metabolite cpd35610 not found in model
1717524070.879058 INFO: cpd07053 not found in model!
1717524070.879851 CRITICAL: Metabolite cpd07053 not found in model
1717524070.880606 INFO: cpd04945 not found in model!
1717524070.881829 CRITICAL: Metabolite cpd04945 not found in model


Lachnospiraceae.5 GF: 13009 Prob: 1771 Other: 3524


1717524076.3135118 INFO: Objective with gapfill database:0.0; min objective:0.01
/Users/chenry/opt/anaconda3/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
1717524081.7425191 INFO: Expansion time:Dlac:0.6200715080003647
1717524081.7430642 INFO: Filtered count:0 out of 1639
1717524082.41303 INFO: Expansion time:Dlac.O2:0.6695755930004452
1717524082.413516 INFO: Filtered count:0 out of 1639
1717524083.07088 INFO: Expansion time:Etho.O2:0.6570354590003262
1717524083.071459 INFO: Filtered count:0 out of 1639
1717524083.698184 INFO: Expansion time:Glc:0.6263041829997746
1717524083.6987321 INFO: Filtered count:0 out of 1639
1717524084.3199842 INFO: Expansion time:Glc.DMSO:0.6209379939991777
1717524084.320537 INFO: Filtered count:0 out of 1639
1717524084.9822469 INFO: Expansion time:Glc.O2:0.6613763049999761
1717524084.982858 INFO: Filtered count:0 out of 1639
1717524085.6278038 INFO: Expan

In [1]:
%run agoracommutil.py
metabolites = util.load("metabolites")
phenotype_results = util.load("phenotype_results")

auxo_media = util.msrecon.get_media("181152/AuxoMedia")
agmm_base_media = util.msrecon.get_media("181152/BaseAnaerobicMM")
gmm_base_media = util.msrecon.get_media("181152/BaseAerobicMM")
uptake_phenoset = util.create_phenotypeset_from_compounds(
    metabolites,
    base_media=auxo_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="uptake"
)

mdlutl = util.msrecon.get_model("Lactobacillus.9.ASV.mdl",181152)
filters = mdlutl.get_attributes("gf_filter")
tests = mdlutl.get_atp_tests(core_template=util.msrecon.core_template,atp_media_filename=util.msrecon.module_dir+"/data/atp_medias.tsv",recompute=False)
msgapfill = MSGapfill(
    mdlutl,
    [util.msrecon.get_template(mdlutl.model.template_ref)],
    [],
    tests,
    blacklist=[],
    default_target="bio1",
    minimum_obj=0.01,
    base_media=None,
    base_media_target_element=None
)
msgapfill.prefilter(test_conditions=tests,growth_conditions=[],use_prior_filtering=True)

#Adding missing transporter for metabolites to gapfilling database
supported_metabolites = {}
for cpd in metabolites:
    if "EX_"+cpd+"_e0" not in msgapfill.gfmodelutl.model.reactions:
        print("Adding transport for ",cpd)
        transport = msgapfill.gfmodelutl.add_transport_and_exchange_for_metabolite(cpd,direction="=",prefix="trans",override=False)
        if transport:
            supported_metabolites[cpd] = transport.id
    else:
        supported_metabolites[cpd] = 1
util.save("supported_metabolites",supported_metabolites)
msgapfill.gfpkgmgr.getpkg("GapfillingPkg").compute_gapfilling_penalties(exclusion_solution=None,reaction_scores=None)

media_list = []
pheno_list = []
target_hash = {}
for pheno in uptake_phenoset.phenotypes:
    if pheno.id in phenotype_results["Lactobacillus.9"]["uptake"]:
        if phenotype_results["Lactobacillus.9"]["uptake"][pheno.id]["objective_value"] < 0.01:
            if pheno.id in supported_metabolites:
                fullmedia = pheno.build_media()
                fullmedia.id = pheno.id
                for item in fullmedia.mediacompounds:
                    if item.id == pheno.id:
                        item.lower_bound = -1000
                        item.upper_bound = -0.01
                pheno_list.append(pheno.id)
                media_list.append(fullmedia)
                #print(phenotype_results["Lactobacillus.9"]["uptake"][pheno.id]["objective"])
                #target_hash[fullmedia] = phenotype_results["Lactobacillus.9"]["uptake"][pheno.id]["objective"]
            else:
                print("No support for ",pheno.id)
phenotype_gf_results = {}
phenotype_gf_results["Lactobacillus.9"] = {}
phenotype_gf_results["Lactobacillus.9"]["uptake"] = msgapfill.run_multi_gapfill(
    media_list,
    #target_hash=target_hash,
    target="bio1",
    default_minimum_objective=0,
    binary_check=False,
    prefilter=False,
    check_for_growth=True,
    gapfilling_mode="Independent",
    run_sensitivity_analysis=False,
    integrate_solutions=False,
    remove_unneeded_reactions=False
)
util.save("phenotype_gf_results",phenotype_gf_results)

python version 3.9.13
KBBaseModules 0.0.1


1717342933.626858 INFO: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
1717342933.62764 INFO: NumExpr defaulting to 8 threads.


modelseedpy 0.3.3
cobrakbase 0.3.1
Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1717342939.0013728 INFO: metabolites 911
1717342939.194977 INFO: reactions 862
1717342939.780438 INFO: Default biomass: [bio1]
1717342939.859582 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.
1717342940.218539 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.


Getting tests from attributes


KeyboardInterrupt: 

In [1]:
import pandas as pd
metabolites = util.load("metabolites")
supported_metabolites = util.load("supported_metabolites")
phenotype_gf_results = util.load("phenotype_gf_results",{})
phenotype_results = util.load("phenotype_results")
asvset_names = util.load("asvset_names")
asvset_genomes = util.load("asvset_genomes")
phenotype_results = util.load("phenotype_results",default={})
phenotype_gf_results = util.load("phenotype_gf_results",default={})
auxo_media = util.msrecon.get_media("181152/AuxoMedia")
agmm_base_media = util.msrecon.get_media("181152/BaseAnaerobicMM")
gmm_base_media = util.msrecon.get_media("181152/BaseAerobicMM")
uptake_phenoset = util.create_phenotypeset_from_compounds(
    columns,
    base_media=auxo_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="uptake"
)
excretion_phenoset = util.create_phenotypeset_from_compounds(
    columns,
    base_media=auxo_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="excretion"
)
agrowth_phenoset = util.create_phenotypeset_from_compounds(
    columns,
    base_media=agmm_base_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="growth"
)
growth_phenoset = util.create_phenotypeset_from_compounds(
    columns,
    base_media=gmm_base_media,
    base_uptake=0,
    base_excretion=1000,
    global_atom_limits={},
    type="growth"
)
for asvset in asvset_genomes:
    asvname = asvset_names[asvset]
    if asvname not in phenotype_results and asvname != "Thermus.1":
        phenotype_gf_results[asvname] = {}
        mdlutl = util.msrecon.get_model(asvname+".ASV.mdl",181152)
        tests = mdlutl.get_atp_tests(core_template=util.msrecon.core_template,atp_media_filename=util.msrecon.module_dir+"/data/atp_medias.tsv",recompute=False)
        msgapfill = MSGapfill(
            mdlutl,
            [util.msrecon.get_template(mdlutl.model.template_ref)],
            [],
            tests,
            blacklist=[],
            default_target="bio1",
            minimum_obj=0.01,
            base_media=None,
            base_media_target_element=None
        )
        msgapfill.prefilter(test_conditions=tests,growth_conditions=[],use_prior_filtering=True)
        #Adding missing transporter for metabolites to gapfilling database
        for cpd in metabolites:
            if "EX_"+cpd+"_e0" not in msgapfill.gfmodelutl.model.reactions:
                print("Adding transport for ",cpd)
                transport = msgapfill.gfmodelutl.add_transport_and_exchange_for_metabolite(cpd,direction="=",prefix="trans",override=False)
        if "uptake" not in phenotype_gf_results[asvname]:
            media_list = []
            pheno_list = []
            target_hash = {}
            for pheno in uptake_phenoset.phenotypes:
                if pheno.id in phenotype_results[asvname]["uptake"]:
                    if phenotype_results[asvname]["uptake"][pheno.id]["objective_value"] < 0.01:
                        if pheno.id in supported_metabolites:
                            fullmedia = pheno.build_media()
                            pheno_list.append(pheno.id)
                            media_list.append(fullmedia)
                            target_hash[fullmedia] = phenotype_results[asvname]["uptake"][pheno.id]["objective"]
            phenotype_gf_results[asvname]["uptake"] = msgapfill.run_multi_gapfill(
                media_list,
                target_hash=target_hash,
                default_minimum_objective=0.01,
                binary_check=False,
                prefilter=False,
                check_for_growth=True,
                gapfilling_mode="Independent",
                run_sensitivity_analysis=False,
                integrate_solutions=False
            )
            util.save("phenotype_gf_results",phenotype_gf_results)
        if "excretion" not in phenotype_gf_results[asvname]:
            media_list = []
            pheno_list = []
            target_hash = {}
            for pheno in uptake_phenoset.phenotypes:
                if pheno.id in phenotype_results[asvname]["excretion"]:
                    if phenotype_results[asvname]["excretion"][pheno.id]["objective_value"] < 0.01:
                        if pheno.id in supported_metabolites:
                            fullmedia = pheno.build_media()
                            pheno_list.append(pheno.id)
                            media_list.append(fullmedia)
                            target_hash[fullmedia] = phenotype_results[asvname]["excretion"][pheno.id]["objective"]
            phenotype_gf_results[asvname]["excretion"] = msgapfill.run_multi_gapfill(
                media_list,
                target_hash=target_hash,
                default_minimum_objective=0.01,
                binary_check=False,
                prefilter=False,
                check_for_growth=True,
                gapfilling_mode="Independent",
                run_sensitivity_analysis=False,
                integrate_solutions=False
            )
            util.save("phenotype_gf_results",phenotype_gf_results)
        if "agrowth" not in phenotype_gf_results[asvname]:
            media_list = []
            pheno_list = []
            target_hash = {}
            for pheno in uptake_phenoset.phenotypes:
                if pheno.id in phenotype_results[asvname]["agrowth"]:
                    if phenotype_results[asvname]["agrowth"][pheno.id]["objective_value"] < 0.01:
                        if pheno.id in supported_metabolites:
                            fullmedia = pheno.build_media()
                            pheno_list.append(pheno.id)
                            media_list.append(fullmedia)
                            target_hash[fullmedia] = phenotype_results[asvname]["agrowth"][pheno.id]["objective"]
            phenotype_gf_results[asvname]["agrowth"] = msgapfill.run_multi_gapfill(
                media_list,
                targe="bio1",
                default_minimum_objective=0.01,
                binary_check=False,
                prefilter=False,
                check_for_growth=True,
                gapfilling_mode="Independent",
                run_sensitivity_analysis=False,
                integrate_solutions=False
            )
            util.save("phenotype_gf_results",phenotype_gf_results)
        if "growth" not in phenotype_gf_results[asvname]:
            media_list = []
            pheno_list = []
            target_hash = {}
            for pheno in uptake_phenoset.phenotypes:
                if pheno.id in phenotype_results[asvname]["growth"]:
                    if phenotype_results[asvname]["growth"][pheno.id]["objective_value"] < 0.01:
                        if pheno.id in supported_metabolites:
                            fullmedia = pheno.build_media()
                            pheno_list.append(pheno.id)
                            media_list.append(fullmedia)
                            target_hash[fullmedia] = phenotype_results[asvname]["growth"][pheno.id]["objective"]
            phenotype_gf_results[asvname]["growth"] = msgapfill.run_multi_gapfill(
                media_list,
                targe="bio1",
                default_minimum_objective=0.01,
                binary_check=False,
                prefilter=False,
                check_for_growth=True,
                gapfilling_mode="Independent",
                run_sensitivity_analysis=False,
                integrate_solutions=False
            )
            util.save("phenotype_gf_results",phenotype_gf_results)

NameError: name 'util' is not defined

# Consolidating gapfilling phenotype data sets 

In [4]:
%run agoracommutil.py
consolidated_gf_results = util.load("consolidated_gf_results")
for i in range(0,8,1):
    phenotype_gf_results = util.load("gf_phenotype_results_"+str(i))
    for asvname in phenotype_gf_results:
        consolidated_gf_results[asvname] = phenotype_gf_results[asvname]
util.save("consolidated_gf_results",consolidated_gf_results)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Computing proper gapfill and filtering bad predictions

In [5]:
%run agoracommutil.py
consolidated_gf_results = util.load("consolidated_gf_results")
feature_probabilities = util.load("feature_probabilities")
studies = ["uptake","excretion","growth"]
for asvname in consolidated_gf_results:
    #Getting base model
    mdlutl = util.msrecon.get_model(asvname+".ASV.mdl",181152)
    
    reaction_probabilities = {}
    for rxn in mdlutl.model.reactions:
        highest_prob = None
        for gene in rxn.genes:
            if gene.id in feature_probabilities:
                if highest_prob == None or feature_probabilities[gene.id] > highest_prob:
                    highest_prob = feature_probabilities[gene.id]
        if highest_prob != None:
            rxn.probability = highest_prob
            reaction_probabilities[rxn.id] = highest_prob
    
    for study in studies:
        if study not in consolidated_gf_results[asvname]:
            continue
        for phenotype in consolidated_gf_results[asvname][study]:
            if "reactions" in consolidated_gf_results[asvname][study][phenotype]:
                consolidated_gf_results[asvname][study][phenotype]["new_probability"] = 0
                consolidated_gf_results[asvname][study][phenotype]["probrxn"] = 0
                consolidated_gf_results[asvname][study][phenotype]["otherrxn"] = 0
                consolidated_gf_results[asvname][study][phenotype]["origgfrxn"] = 0
                consolidated_gf_results[asvname][study][phenotype]["new_gf"] = 0
                consolidated_gf_results[asvname][study][phenotype]["new_gf_reactions"] = []
                found = False
                for rxn in consolidated_gf_results[asvname][study][phenotype]["reactions"]:
                    direction = rxn[0:1]
                    rxnid = rxn[1:]
                    if rxnid[0:3] == "EX_" and rxnid[3:11] == phenotype:
                        found = True
                    if rxnid not in mdlutl.model.reactions:
                        consolidated_gf_results[asvname][study][phenotype]["new_gf_reactions"].append(rxn)
                        consolidated_gf_results[asvname][study][phenotype]["new_gf"] += 1
                    elif direction == ">":
                        if mdlutl.model.reactions.get_by_id(rxnid).upper_bound <= 0:
                            consolidated_gf_results[asvname][study][phenotype]["new_gf_reactions"].append(rxn)
                            consolidated_gf_results[asvname][study][phenotype]["new_gf"] += 1
                        elif rxnid in reaction_probabilities:
                            consolidated_gf_results[asvname][study][phenotype]["new_probability"] += reaction_probabilities[rxnid]
                            consolidated_gf_results[asvname][study][phenotype]["probrxn"] += 1
                        elif rxnid[0:3] != "bio" and rxnid[0:3] != "EX_" and rxnid[0:3] != "DM_" and len(mdlutl.model.reactions.get_by_id(rxnid).genes) == 0:
                            consolidated_gf_results[asvname][study][phenotype]["origgfrxn"] += 1
                        else:
                            consolidated_gf_results[asvname][study][phenotype]["otherrxn"] += 1
                            consolidated_gf_results[asvname][study][phenotype]["new_probability"] += 0.05  
                    elif direction == "<":
                        if mdlutl.model.reactions.get_by_id(rxnid).lower_bound >= 0:
                            consolidated_gf_results[asvname][study][phenotype]["new_gf_reactions"].append(rxn)
                            consolidated_gf_results[asvname][study][phenotype]["new_gf"] += 1
                        elif rxnid in reaction_probabilities:
                            consolidated_gf_results[asvname][study][phenotype]["new_probability"] += reaction_probabilities[rxnid]
                            consolidated_gf_results[asvname][study][phenotype]["probrxn"] += 1
                        elif rxnid[0:3] != "bio" and rxnid[0:3] != "EX_" and rxnid[0:3] != "DM_" and len(mdlutl.model.reactions.get_by_id(rxnid).genes) == 0:
                            consolidated_gf_results[asvname][study][phenotype]["origgfrxn"] += 1
                        else:
                            consolidated_gf_results[asvname][study][phenotype]["otherrxn"] += 1
                            consolidated_gf_results[asvname][study][phenotype]["new_probability"] += 0.05  
                totalrxn = consolidated_gf_results[asvname][study][phenotype]["probrxn"]+consolidated_gf_results[asvname][study][phenotype]["new_gf"]+consolidated_gf_results[asvname][study][phenotype]["otherrxn"]
                consolidated_gf_results[asvname][study][phenotype]["new_probability"] = consolidated_gf_results[asvname][study][phenotype]["new_probability"]/totalrxn

                if not found:
                    consolidated_gf_results[asvname][study][phenotype]["original_objective_value"] = consolidated_gf_results[asvname][study][phenotype]["objective_value"]
                    consolidated_gf_results[asvname][study][phenotype]["objective_value"] = 0
                    consolidated_gf_results[asvname][study][phenotype]["class"] = "N"
                    consolidated_gf_results[asvname][study][phenotype]["positive"] = False
                    consolidated_gf_results[asvname][study][phenotype]["new_probability"] = 0
util.save("consolidated_gf_results",consolidated_gf_results)  
    

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1717559851.019614 INFO: metabolites 1279
1717559851.367193 INFO: reactions 1274
1717559852.094133 INFO: Default biomass: [bio1]
1717559853.308639 INFO: metabolites 1523
1717559853.70419 INFO: reactions 1661
1717559854.5291111 INFO: Default biomass: [bio1]
1717559856.665163 INFO: metabolites 1674
1717559857.085002 INFO: reactions 1843
1717559858.064928 INFO: Default biomass: [bio1]
1717559859.0516322 INFO: metabolites 1169
1717559859.29213 INFO: reactions 1142
1717559859.861557 INFO: Default biomass: [bio1]
1717559860.689198 INFO: metabolites 1236
1717559860.9425569 INFO: reactions 1251
1717559861.546896 INFO: Default biomass: [bio1]
1717559862.408082 INFO: metabolites 1159
1717559862.6575959 INFO: reactions 1226
1717559863.258187 INFO: Default biomass: [bio1]
1717559864.281517 INFO: metabolites 1084
1717559864.4800808 INFO: reactions 1056
1717559864.989701 INFO: Default biomass: [bio1]
1717559866.405629 INFO: metabolites 1292
1717559866.69237 INFO: reactions 1375
1717559867.356698 INFO

# Printing matrices

In [6]:
%run agoracommutil.py
consolidated_gf_results = util.load("consolidated_gf_results")

asv_asvset = util.load("asv_asvset")
metabolites = util.load("metabolites")
metabolite_names = util.load("metabolite_names")
phenotype_results = util.load("phenotype_results")
asvset_genomes = util.load("asvset_genomes")
asvset_names = util.load("asvset_names")
asvset_abundances = util.load("asvset_abundances")
sample_asvset_abundances = util.load("sample_asvset_abundances")
sample_abundanes = util.load("sample_abundanes")
records = {
    "uptake_prob":[{"Sample":"Name"}],
    "excretion_prob":[{"Sample":"Name"}],
    "growth_prob":[{"Sample":"Name"}],
}
types = ["uptake","excretion","growth"]
for met in metabolites:
    for item in records:
        records[item][0][met] = metabolite_names[met]
for sample in sample_asvset_abundances:
    new_record = {
        "uptake_prob":{"Sample":sample},
        "excretion_prob":{"Sample":sample},
        "growth_prob":{"Sample":sample}
    }
    for metabolite in metabolites:
        for item in new_record:
            new_record[item][metabolite] = 0
    for asvset in sample_asvset_abundances[sample]:
        asvname = asvset_names[asvset]
        for metabolite in metabolites:
            for currtype in types:
                if currtype in consolidated_gf_results[asvname]:
                    if metabolite in consolidated_gf_results[asvname][currtype] and "new_probability" in consolidated_gf_results[asvname][currtype][metabolite]:
                        new_record[currtype+"_prob"][metabolite] += sample_asvset_abundances[sample][asvset]/sample_abundanes[sample]["asvset_total"]*consolidated_gf_results[asvname][currtype][metabolite]["new_probability"]
    for item in records:
        records[item].append(new_record[item])
#Printing interation matrices
for item in records:
    df = pd.DataFrame.from_records(records[item])
    df.to_csv(util.output_dir+"/Sample"+item+"Interactions.csv",index=False)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Printing ASV data

In [7]:
%run agoracommutil.py
consolidated_gf_results = util.load("consolidated_gf_results")

asv_asvset = util.load("asv_asvset")
metabolites = util.load("metabolites")
metabolite_names = util.load("metabolite_names")
phenotype_results = util.load("phenotype_results")
asvset_genomes = util.load("asvset_genomes")
asvset_names = util.load("asvset_names")
asvset_abundances = util.load("asvset_abundances")
sample_asvset_abundances = util.load("sample_asvset_abundances")
sample_abundanes = util.load("sample_abundanes")
records = {
    "uptake_prob":[{"ASV":"Name"}],
    "excretion_prob":[{"ASV":"Name"}],
    "growth_prob":[{"ASV":"Name"}],
    "uptake_data":[{"ASV":"Name"}],
    "excretion_data":[{"ASV":"Name"}],
    "growth_data":[{"ASV":"Name"}],
}
types = ["uptake","excretion","growth"]
for met in metabolites:
    for item in records:
        records[item][0][met] = metabolite_names[met]
for asvset in asvset_genomes:
    asvname = asvset_names[asvset]
    if asvname not in consolidated_gf_results:
        print("No data for ",asvname)
        continue
    new_record = {}
    for currtype in types:
        new_record[currtype+"_prob"] = {"ASV":asvname}
        new_record[currtype+"_data"] = {"ASV":asvname}
    for metabolite in metabolites:
        for currtype in types:
            if currtype in consolidated_gf_results[asvname]:
                if metabolite in consolidated_gf_results[asvname][currtype] and "new_probability" in consolidated_gf_results[asvname][currtype][metabolite]:
                    new_record[currtype+"_prob"][metabolite] = consolidated_gf_results[asvname][currtype][metabolite]["new_probability"]
                    new_record[currtype+"_data"][metabolite] = str(consolidated_gf_results[asvname][currtype][metabolite]["probrxn"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[asvname][currtype][metabolite]["otherrxn"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[asvname][currtype][metabolite]["origgfrxn"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[asvname][currtype][metabolite]["new_gf"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[asvname][currtype][metabolite]["gapfill_count"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[asvname][currtype][metabolite]["reaction_count"])
    for item in records:
        records[item].append(new_record[item])
#Printing interation matrices
for item in records:
    df = pd.DataFrame.from_records(records[item])
    df.to_csv(util.output_dir+"/ASV"+item+"Interactions.csv",index=False)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/
No data for  Thermus.1


# Gapfilling models based on gapfilling simulation results

In [3]:
%run agoracommutil.py
import numpy as np
consolidated_gf_results = util.load("consolidated_gf_results")
finished_models = util.load("finished_models",{})
metabolites = util.load("metabolites")
reactions_to_add = {}

for asvname in consolidated_gf_results:
    if asvname not in finished_models:
        mdlutl = util.msrecon.get_model(asvname+".ASV.mdl",181152)
        types = ["uptake","excretion","growth"]
        reactions_to_add[asvname] = {}
        for currtype in types:
            problist = []
            for metabolite in metabolites:
                if metabolite in consolidated_gf_results[asvname][currtype] and "new_probability" in consolidated_gf_results[asvname][currtype][metabolite]:
                    if consolidated_gf_results[asvname][currtype][metabolite]["new_probability"] > 0:
                        problist.append(consolidated_gf_results[asvname][currtype][metabolite]["new_probability"])
            if len(problist) > 0:
                ave = np.array(problist).mean()
                stdev = np.array(problist).std()
                for metabolite in metabolites:
                    if metabolite in consolidated_gf_results[asvname][currtype] and "new_probability" in consolidated_gf_results[asvname][currtype][metabolite]:
                        if consolidated_gf_results[asvname][currtype][metabolite]["new_probability"] > 0:
                            consolidated_gf_results[asvname][currtype][metabolite]["zscore"] = (consolidated_gf_results[asvname][currtype][metabolite]["new_probability"]-ave)/stdev
                            if consolidated_gf_results[asvname][currtype][metabolite]["zscore"] >= -1 and "reactions" in consolidated_gf_results[asvname][currtype][metabolite]:
                                for rxn in consolidated_gf_results[asvname][currtype][metabolite]["reactions"]:
                                    direction = rxn[0:1]
                                    rxnid = rxn[1:]
                                    if rxnid not in mdlutl.model.reactions:
                                        if rxnid not in reactions_to_add[asvname]:
                                            reactions_to_add[asvname][rxnid] = {}
                                        reactions_to_add[asvname][rxnid][direction] = consolidated_gf_results[asvname][currtype][metabolite]["zscore"]
                                    elif direction == ">":
                                        if mdlutl.model.reactions.get_by_id(rxnid).upper_bound <= 0:
                                            if rxnid not in reactions_to_add[asvname]:
                                                reactions_to_add[asvname][rxnid] = {}
                                            reactions_to_add[asvname][rxnid][direction] = consolidated_gf_results[asvname][currtype][metabolite]["zscore"]
                                    elif direction == "<":
                                        if mdlutl.model.reactions.get_by_id(rxnid).lower_bound >= 0:
                                            if rxnid not in reactions_to_add[asvname]:
                                                reactions_to_add[asvname][rxnid] = {}
                                            reactions_to_add[asvname][rxnid][direction] = consolidated_gf_results[asvname][currtype][metabolite]["zscore"]
        util.save("consolidated_gf_results",consolidated_gf_results)
        msgapfill = MSGapfill(
            mdlutl,
            [util.msrecon.get_template(mdlutl.model.template_ref)],
            [],
            [],
            blacklist=[],
            default_target="bio1",
            minimum_obj=0.01,
            base_media=None,
            base_media_target_element=None
        )
        for cpd in metabolites:
            if "EX_"+cpd+"_e0" not in msgapfill.gfmodelutl.model.reactions:
                transport = msgapfill.gfmodelutl.add_transport_and_exchange_for_metabolite(cpd,direction="=",prefix="trans",override=False)
        for rxnid in reactions_to_add[asvname]:
            if rxnid in msgapfill.gfmodel.reactions:
                rxn = msgapfill.gfmodel.reactions.get_by_id(rxnid)
                rxn = rxn.copy()
                mdlutl.model.add_reactions([rxn])
                rxn.upper_bound = 0
                rxn.lower_bound = 0
                if ">" in reactions_to_add[asvname][rxnid]:
                    rxn.upper_bound = 100
                if "<" in reactions_to_add[asvname][rxnid]:
                    rxn.lower_bound = -100
            else:
                print(asvname,"Missing reaction",rxnid,reactions_to_add[asvname][rxnid])
        util.msrecon.save_model(mdlutl,181152,asvname+".ASV.mdl.gf")
        finished_models[asvname] = 1
    util.save("finished_models",finished_models)
util.save("reactions_to_add",reactions_to_add)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1717567041.530565 INFO: metabolites 1236
1717567041.790891 INFO: reactions 1251
1717567042.434616 INFO: Default biomass: [bio1]
1717567058.6264582 INFO: cpd35610 not found in model!
1717567058.627019 CRITICAL: Metabolite cpd35610 not found in model
1717567058.643171 INFO: cpd07053 not found in model!
1717567058.64384 CRITICAL: Metabolite cpd07053 not found in model
1717567058.644323 INFO: cpd04945 not found in model!
1717567058.64481 CRITICAL: Metabolite cpd04945 not found in model
1717567058.7683961 WARNING: Ignoring reaction 'rxn09180_c0' since it already exists.
1717567058.813857 WARNING: Ignoring reaction 'rxn05561_c0' since it already exists.
1717567058.969562 INFO: Setting FBAModel computed_attributes to mdlutl attributes


Enterococcus.2 Missing reaction EX_cpd00557_c0 {'>': 0.12320732152100093}


1717567061.624893 INFO: metabolites 1159
1717567061.865946 INFO: reactions 1226
1717567062.496444 INFO: Default biomass: [bio1]
1717567077.600456 INFO: cpd35610 not found in model!
1717567077.601034 CRITICAL: Metabolite cpd35610 not found in model
1717567077.616554 INFO: cpd07053 not found in model!
1717567077.617491 CRITICAL: Metabolite cpd07053 not found in model
1717567077.618251 INFO: cpd04945 not found in model!
1717567077.61945 CRITICAL: Metabolite cpd04945 not found in model


Corynebacterium.2 Missing reaction EX_cpd00557_c0 {'>': 0.7965662211201041}


...

1717572515.747434 WARNING: Ignoring reaction 'rxn10663_c0' since it already exists.
1717572515.7492008 WARNING: Ignoring reaction 'rxn00945_c0' since it already exists.
1717572515.796032 WARNING: Ignoring reaction 'rxn01045_c0' since it already exists.
1717572515.809043 INFO: Setting FBAModel computed_attributes to mdlutl attributes
1717572518.5385559 INFO: metabolites 873
1717572518.6724741 INFO: reactions 831
1717572519.0437949 INFO: Default biomass: [bio1]
1717572533.1990452 INFO: cpd35610 not found in model!
1717572533.1996958 CRITICAL: Metabolite cpd35610 not found in model
1717572533.2140648 INFO: cpd07053 not found in model!
1717572533.214806 CRITICAL: Metabolite cpd07053 not found in model
1717572533.215224 INFO: cpd04945 not found in model!
1717572533.215612 CRITICAL: Metabolite cpd04945 not found in model
1717572533.222783 WARNING: Ignoring reaction 'rxn00430_c0' since it already exists.
1717572533.3050282 WARNING: Ignoring reaction 'rxn05299_c0' since it already exists.
1717

Butyrivibrio.1 Missing reaction EX_cpd00557_c0 {'>': 1.3086631627189294}


1717572533.595659 WARNING: Ignoring reaction 'rxn00350_c0' since it already exists.
1717572533.651677 WARNING: Ignoring reaction 'rxn00023_c0' since it already exists.
1717572533.665389 WARNING: Ignoring reaction 'rxn00272_c0' since it already exists.
1717572533.787759 WARNING: Ignoring reaction 'rxn10124_c0' since it already exists.
1717572533.790512 INFO: Setting FBAModel computed_attributes to mdlutl attributes
1717572537.0926452 INFO: metabolites 1147
1717572537.2894342 INFO: reactions 1139
1717572537.771297 INFO: Default biomass: [bio1]
1717572550.819402 INFO: cpd35610 not found in model!
1717572550.820012 CRITICAL: Metabolite cpd35610 not found in model
1717572550.835798 INFO: cpd07053 not found in model!
1717572550.836875 CRITICAL: Metabolite cpd07053 not found in model
1717572550.8378801 INFO: cpd04945 not found in model!
1717572550.839 CRITICAL: Metabolite cpd04945 not found in model
1717572551.0348392 WARNING: Ignoring reaction 'rxn00502_c0' since it already exists.
171757255

Vallitalea.1 Missing reaction EX_cpd00557_c0 {'>': 0.38243312652181716}


1717572568.7897449 INFO: Setting FBAModel computed_attributes to mdlutl attributes


# Building and saving community model

In [2]:
%run agoracommutil.py
import cobra
sample_asvset_abundances = util.load("sample_asvset_abundances")
sample_abundanes = util.load("sample_abundanes")
asvset_names = util.load("asvset_names")

sample_pairs = [
    ["RC-ABX_-3","RC-ABX_0","RC-ABX_-1.5"],
    ["RC-ABX_0","RC-ABX_3","RC-ABX_1.5"],
    ["RC-ABX_3","RC-ABX_5","RC-ABX_4.0"],
    ["RC-ABX_5","RC-ABX_7","RC-ABX_6.0"],
    ["RC-ABX_8","RC-ABX_11","RC-ABX_9.0"],
    ["RC-ABX_11", "RC-ABX_13","RC-ABX_12.5"]
]
for pair in sample_pairs:
    member_models = []
    member_names = []
    member_abundances = {}
    asv_hash = {}
    for asvset in sample_asvset_abundances[pair[0]]:
        asvname = asvset_names[asvset]
        if asvname not in asv_hash:
            asv_hash[asvname] = sample_asvset_abundances[pair[0]][asvset]
        else:
            asv_hash[asvname] += sample_asvset_abundances[pair[0]][asvset]
    for asvset in sample_asvset_abundances[pair[1]]:
        asvname = asvset_names[asvset]
        if asvname not in asv_hash:
            asv_hash[asvname] = sample_asvset_abundances[pair[1]][asvset]
        else:
            asv_hash[asvname] += sample_asvset_abundances[pair[1]][asvset]
    total_abundance = 0
    for asvname in asv_hash:
        asv_hash[asvname] = asv_hash[asvname]/2
        total_abundance += asv_hash[asvname]
    for asvname in asv_hash:
        asv_hash[asvname] = asv_hash[asvname]/total_abundance
        if asvname != "Thermus.1" and asv_hash[asvname] > 0.01:
            mdlutl = util.msrecon.get_model(asvname+".ASV.mdl.gf",181152)
            member_models.append(mdlutl.model)
            member_names.append(asvname)
            member_abundances[asvname] = asv_hash[asvname]
    comm_model = MSCommunity.build_from_species_models(
        member_models,
        mdlid=pair[2], 
        name=pair[2],
        names=member_names,
        abundances=member_abundances
    )
    cobra.io.save_json_model(comm_model.model, "models/"+pair[2]+'.json')

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1717728513.6705658 INFO: metabolites 1482
1717728513.972935 INFO: reactions 1616
1717728514.7140841 INFO: Default biomass: [bio1]
1717728515.457014 INFO: metabolites 1023
1717728515.616482 INFO: reactions 1118
1717728516.119099 INFO: Default biomass: [bio1]
1717728516.946501 INFO: metabolites 1060
1717728517.112511 INFO: reactions 1090
1717728518.286844 INFO: Default biomass: [bio1]
1717728519.650422 INFO: metabolites 1374
1717728519.906087 INFO: reactions 1490
1717728520.591357 INFO: Default biomass: [bio1]
1717728521.4609768 INFO: metabolites 1058
1717728521.622852 INFO: reactions 1169
1717728522.142626 INFO: Default biomass: [bio1]
1717728522.742181 INFO: metabolites 1286
1717728522.9630668 INFO: reactions 1365
1717728523.569077 INFO: Default biomass: [bio1]
1717728524.4064698 INFO: metabolites 1147
1717728524.582432 INFO: reactions 1210
1717728525.1070879 INFO: Default biomass: [bio1]
1717728526.7064378 INFO: metabolites 1294
1717728526.946501 INFO: reactions 1397
1717728527.571569

In [3]:
%run agoracommutil.py
import cobra
sample_asvset_abundances = util.load("sample_asvset_abundances")
sample_abundanes = util.load("sample_abundanes")
asvset_names = util.load("asvset_names")

sample_pairs = [
    ["RC-ABX_-3","RC-ABX_0","RC-ABX_-1.5"],
    ["RC-ABX_0","RC-ABX_3","RC-ABX_1.5"],
    ["RC-ABX_3","RC-ABX_5","RC-ABX_4.0"],
    ["RC-ABX_5","RC-ABX_7","RC-ABX_6.0"],
    ["RC-ABX_8","RC-ABX_11","RC-ABX_9.0"],
    ["RC-ABX_11", "RC-ABX_13","RC-ABX_12.5"]
]

community_model_data = {}
for pair in sample_pairs:
    member_models = []
    member_names = []
    member_abundances = {}
    asv_hash = {}
    for asvset in sample_asvset_abundances[pair[0]]:
        asvname = asvset_names[asvset]
        if asvname not in asv_hash:
            asv_hash[asvname] = sample_asvset_abundances[pair[0]][asvset]
        else:
            asv_hash[asvname] += sample_asvset_abundances[pair[0]][asvset]
    for asvset in sample_asvset_abundances[pair[1]]:
        asvname = asvset_names[asvset]
        if asvname not in asv_hash:
            asv_hash[asvname] = sample_asvset_abundances[pair[1]][asvset]
        else:
            asv_hash[asvname] += sample_asvset_abundances[pair[1]][asvset]
    total_abundance = 0
    for asvname in asv_hash:
        asv_hash[asvname] = asv_hash[asvname]/2
        total_abundance += asv_hash[asvname]
    for asvname in asv_hash:
        asv_hash[asvname] = asv_hash[asvname]/total_abundance
        if asvname != "Thermus.1" and asv_hash[asvname] > 0.01:
            member_names.append(asvname)
            member_abundances[asvname] = asv_hash[asvname]
    community_model_data[pair[2]] = {
        "names":member_names,
        "abundances":member_abundances
    }
    util.save("community_model_data",community_model_data)
    

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Simulating community model in each condition

In [1]:
%run agoracommutil.py
procid = 0
metabolites = util.load("metabolites")
metabolomics_data = util.load("metabolomics_data")
feature_probabilities = util.load("feature_probabilities")
community_model_data = util.load("community_model_data")
from optlang.symbolics import Zero, add
import cobra

complete_media = util.msrecon.get_media("KBaseMedia/Complete")

model_list = ["RC-ABX_-1.5","RC-ABX_1.5","RC-ABX_4.0","RC-ABX_6.0","RC-ABX_9.0","RC-ABX_12.5"]
output = {}
rxn_record_hash = {}
records = [
    {"id":"max_growth"},
    {"id":"carbon_uptake"},
    {"id":"flux fitting objective"},
    {"id":"minimum probability objective"},
]
for modelid in model_list:
    output[modelid] = {}
    base_model = cobra.io.load_json_model("models/"+modelid+'.json')
    current_comm_model = MSCommunity(
        model=base_model,
        names=community_model_data[modelid]["names"]
    )
    #loadedmodel.solver = 'gurobi'
    dipeptide_exchanges = [
        "EX_cpd11591_e0",
        "EX_cpd11589_e0",
        "EX_cpd15605_e0",
        "EX_cpd11588_e0",
        "EX_cpd11583_e0",
        "EX_cpd11580_e0",
        "EX_cpd11593_e0",
        "EX_cpd11585_e0",
        "EX_cpd11586_e0",
        "EX_cpd15604_e0",
        "EX_cpd11581_e0",
        "EX_cpd01017_e0",
        "EX_cpd11590_e0",
        "EX_cpd11592_e0",
        "EX_cpd11584_e0",
        "EX_cpd00731_e0",
        "EX_cpd15603_e0",
        "EX_cpd11587_e0",
        "EX_cpd11582_e0",
        "EX_cpd15606_e0",
        "EX_cpd03424_e0",
        "EX_cpd00423_e0",
        "EX_cpd00080_e0",
        "EX_cpd02233_e0",
        "EX_cpd00355_e0",
        "EX_cpd00235_e0",
        "EX_cpd00079_e0",
        "EX_cpd01912_e0"
    ]
    reaction_probabilities = {}
    for rxn in base_model.reactions:
        highest_prob = None
        for gene in rxn.genes:
            if gene.id in feature_probabilities:
                if highest_prob == None or feature_probabilities[gene.id] > highest_prob:
                    highest_prob = feature_probabilities[gene.id]
        if highest_prob != None:
            rxn.probability = highest_prob
            reaction_probabilities[rxn.id] = highest_prob
        elif rxn.id[0:3] != "bio" and rxn.id[0:3] != "EX_" and rxn.id[0:3] != "DM_" and len(rxn.genes) == 0:
            reaction_probabilities[rxn.id] = 0
        else:
            reaction_probabilities[rxn.id] = 0.05
    min_prob = 0.05
    prob_exp = 1
    ex_weight = 1
    kinetics_coef = 750
    mdlutl = current_comm_model.mdlutl
    pkgmgr = MSPackageManager.get_pkg_mgr(mdlutl)
    #Setting media
    pkgmgr.getpkg("KBaseMediaPkg").build_package(complete_media)
    #Adding commkinetic constraints
    pkgmgr.getpkg("CommKineticPkg").build_package(kinetics_coef, current_comm_model)
    #Adding elemental uptake constraints
    pkgmgr.getpkg("ElementUptakePkg").build_package({"C": 300})
    for rxn in base_model.reactions:
        if "EX_" == rxn.id[0:3]:
            currrxn = current_comm_model.model.reactions.get_by_id(rxn.id)
            if rxn.id in dipeptide_exchanges:
                currrxn.lower_bound = 0
                currrxn.upper_bound = 0
            else:
                if currrxn.lower_bound < 0:
                    currrxn.lower_bound = -1000
                if currrxn.upper_bound > 0:
                    currrxn.upper_bound = 1000
    currrxn = current_comm_model.model.reactions.get_by_id("EX_cpd00007_e0")
    currrxn.lower_bound = -20
    #Maximize biomass production
    mdlutl.model.objective = mdlutl.model.problem.Objective(Zero, direction="max")
    mdlutl.model.objective.set_linear_coefficients({mdlutl.model.reactions.bio1.forward_variable: 1})
    first_solution = mdlutl.model.optimize()
    output[modelid]["max_growth"] = first_solution.objective_value
    output[modelid]["carbon_uptake"] = pkgmgr.getpkg("ElementUptakePkg").variables["elements"]["C"].primal
    records[0][modelid] = output[modelid]["max_growth"]
    records[1][modelid] = output[modelid]["carbon_uptake"]
    print(modelid+" growth:", output[modelid]["max_growth"])
    print(modelid+"carbon uptake:",output[modelid]["carbon_uptake"])
    
    with open(util.output_dir+"/"+modelid+"-growth.lp", "w") as out:
        out.write(str(mdlutl.model.solver))

    if str(output[modelid]["max_growth"]) == "nan":
        print("Skipping condition due to infeasibility", modelid)
        continue
    #Constraining to 70% of community biomass
    pkgmgr.getpkg("ObjConstPkg").build_package(output[modelid]["max_growth"] * 0.5, None)
    #Creating minimal probability objective
    coef = {}
    flux_fitting_target = {}
    for rxn in base_model.reactions:
        if "rxn" == rxn.id[0:3]:
            probability = reaction_probabilities[rxn.id]
            currrxn = mdlutl.model.reactions.get_by_id(rxn.id)
            coef.update(
                {
                    currrxn.forward_variable: max(
                        min_prob, (1 - float(probability) ** prob_exp)
                    )
                }
            )
            coef.update(
                {
                    currrxn.reverse_variable: max(
                        min_prob, (1 - float(probability) ** prob_exp)
                    )
                }
            )
        elif "EX_" == rxn.id[0:3]:
            currrxn = mdlutl.model.reactions.get_by_id(rxn.id)
            if rxn.id[3:11] in metabolomics_data:
                flux_fitting_target[rxn.id] = -1*output[modelid]["max_growth"]*metabolomics_data[rxn.id[3:11]][modelid]
            coef.update({currrxn.forward_variable: ex_weight})
            coef.update({currrxn.reverse_variable: ex_weight})

    #Solving the LP
    #pkgmgr.getpkg("FluxFittingPkg").build_package({
    #    "target_flux": flux_fitting_target,
    #    "totalflux": 0,
    #    "set_objective": 1,
    #    "default_rescaling": 0.1,
    #    "rescale_vfit_by_flux": True
    #})
    stuck_reactions = {}
    for iteration in range(1,11,1):
        print("Iteration",iteration)
        for rxn in flux_fitting_target:
            if rxn in stuck_reactions:
                continue
            currrxn = mdlutl.model.reactions.get_by_id(rxn)
            starting_point = first_solution.fluxes[rxn]
            distance = abs(flux_fitting_target[rxn] - starting_point)
            original_upper = currrxn.upper_bound
            original_lower = currrxn.lower_bound
            if starting_point > flux_fitting_target[rxn]:
                if starting_point-iteration*distance/10 < currrxn.upper_bound:
                    currrxn.lower_bound = starting_point-iteration*distance/10
                    currrxn.upper_bound = starting_point-iteration*distance/10
                else:
                    currrxn.upper_bound = starting_point-iteration*distance/10
                    currrxn.lower_bound = starting_point-iteration*distance/10
            else:
                if flux_fitting_target[rxn]-(10-iteration)*distance/10 < currrxn.upper_bound:
                    currrxn.lower_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
                    currrxn.upper_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
                else:
                    currrxn.upper_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
                    currrxn.lower_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
            solution = mdlutl.model.optimize()
            if solution.status != "optimal":
                print("Stuck",rxn,starting_point,flux_fitting_target[rxn],original_upper)
                stuck_reactions[rxn] = 1
                if original_upper >= currrxn.lower_bound:
                    currrxn.upper_bound = original_upper
                    currrxn.lower_bound = original_lower
                else:
                    currrxn.lower_bound = original_lower
                    currrxn.upper_bound = original_upper
    for rxn in flux_fitting_target:
        currrxn = mdlutl.model.reactions.get_by_id(rxn)
        print(currrxn.id,currrxn.lower_bound,currrxn.upper_bound)
    #solution = mdlutl.model.optimize()
    #output[modelid]["flux fitting objective"] = solution.objective_value
    
    #with open(util.output_dir+"/"+modelid+"-fitting.lp", "w") as out:
    #    out.write(str(mdlutl.model.solver))

    #records[2][modelid] = output[modelid]["flux fitting objective"]
    #for rxn in flux_fitting_target:
    #    currrxn = current_comm_model.model.reactions.get_by_id(rxn)
    #    if solution.fluxes[rxn] > 0:
    #        currrxn.upper_bound = solution.fluxes[rxn]
    #        currrxn.lower_bound = solution.fluxes[rxn]
    #    elif solution.fluxes[rxn] < 0:
    #        currrxn.lower_bound = solution.fluxes[rxn]
    #        currrxn.upper_bound = solution.fluxes[rxn]
    #Setting probability minimization objective
    mdlutl.model.objective = mdlutl.model.problem.Objective(Zero, direction="min")
    mdlutl.model.objective.set_linear_coefficients(coef)
    solution = mdlutl.model.optimize()
    if solution.status != "optimal":
        while solution.status != "optimal":
            print("Infeasible: reducing objective constraint",0.9*mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"]["1"].lb)
            mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"]["1"].lb = 0.9*mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"]["1"].lb
            solution = mdlutl.model.optimize()
    output[modelid]["minimum probability objective"] = solution.objective_value
    
    with open(util.output_dir+"/"+modelid+"-final.lp", "w") as out:
        out.write(str(mdlutl.model.solver))

    records[3][modelid] = output[modelid]["minimum probability objective"]
    output[modelid]["solution"] = {}
    for rxn in mdlutl.model.reactions:
        if rxn.id not in rxn_record_hash:
            rxn_record_hash[rxn.id] = {
                "id":rxn.id,
            }
            for item in model_list:
                rxn_record_hash[rxn.id][item] = None
            records.append(rxn_record_hash[rxn.id])
        rxn_record_hash[rxn.id][modelid] = solution.fluxes[rxn.id]
        output[modelid]["solution"][rxn.id] = solution.fluxes[rxn.id]
    util.save("CommunityFluxSolution2", output)
    df = DataFrame.from_records(records)
    df.to_csv(util.output_dir+"/CommunityFluxSolution2.csv")

python version 3.9.13
KBBaseModules 0.0.1


1717943223.189431 INFO: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
1717943223.190034 INFO: NumExpr defaulting to 8 threads.


modelseedpy 0.3.3
cobrakbase 0.3.1
Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1717943251.4168189 INFO: Making atp hydrolysis reaction for species: Lactobacillus.3
1717943252.043584 INFO: Making atp hydrolysis reaction for species: Merdibacter.1
1717943252.363841 INFO: Making atp hydrolysis reaction for species: Nosocomiicoccus.1
1717943252.677865 INFO: Making atp hydrolysis reaction for species: Staphylococcus.3
1717943253.002086 INFO: Making atp hydrolysis reaction for species: Turicibacter.1
1717943253.3384378 INFO: Making atp hydrolysis reaction for species: Lactobacillus.1
1717943253.959151 INFO: Making atp hydrolysis reaction for species: Muribaculum.1
1717943254.283909 INFO: Making atp hydrolysis reaction for species: Lactobacillus.4
1717943254.607856 INFO: Making atp hydrolysis reaction for species: Clostridium.7
1717943254.930045 INFO: Making atp hydrolysis reaction for species: Alloiococcus.1
1717943255.2549372 INFO: Making atp hydrolysis reaction for species: Lactobacillus.5
1717943255.5946188 INFO: Making atp hydrolysis reaction for species: Desulfovi

RC-ABX_-1.5 growth: 8.445031644486718
RC-ABX_-1.5carbon uptake: 300.0
Iteration 1


...

Stuck EX_cpd00060_e0 -0.14131647526793326 -5.435748287794131 -3.8474187440362715


/Users/chenry/opt/anaconda3/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Stuck EX_cpd00161_e0 0.0 -10.234236470228472 -7.163965529159931
Iteration 9
Iteration 10


/Users/chenry/opt/anaconda3/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Stuck EX_cpd00121_e0 0.0 -2.0693505273833055 -1.862415474644975
EX_cpd00084_e0 0.2791404833829396 0.2791404833829396
EX_cpd00119_e0 -3.017275830657105 -3.017275830657105
EX_cpd00033_e0 0.9701521381703175 0.9701521381703175
EX_cpd01242_e0 -0.25238911010875775 -0.25238911010875775
EX_cpd00137_e0 -0.7493113596734522 -0.7493113596734522
EX_cpd00129_e0 -0.4868421060401726 -0.4868421060401726
EX_cpd00158_e0 -0.2485500136794939 -0.2485500136794939
EX_cpd01171_e0 -2.532102855495168 -2.532102855495168
EX_cpd01107_e0 1.7597483467814312 1.7597483467814312
EX_cpd03847_e0 1.3685083706807974 1.3685083706807974
EX_cpd00322_e0 -1.2408171559494292 -1.2408171559494292
EX_cpd00108_e0 0.7448997069928733 0.7448997069928733
EX_cpd00132_e0 -0.9390873696857083 -0.9390873696857083
EX_cpd00851_e0 0.9911053045895954 0.9911053045895954
EX_cpd02733_e0 1.5368818532152702 1.5368818532152702
EX_cpd00224_e0 1.9428631613069527 1.9428631613069527
EX_cpd00130_e0 1.1154626027409815 1.1154626027409815
EX_cpd00526_e0 -1000 

# Printing ASV results for analysis

In [9]:
%run agoracommutil.py
community_model_data = util.load("community_model_data")
CommunityFluxSolution = util.load("CommunityFluxSolution2")
sample_asvset_abundances = util.load("sample_asvset_abundances")
sample_abundanes = util.load("sample_abundanes")
metabolomics_data = util.load("metabolomics_data")
asvset_names = util.load("asvset_names")
metabolites = util.load("metabolites")
metabolite_names = util.load("metabolite_names")

sample_pairs = {
    "RC-ABX_-1.5" : ["RC-ABX_-3","RC-ABX_0"],
    "RC-ABX_1.5" : ["RC-ABX_0","RC-ABX_3"],
    "RC-ABX_4.0" : ["RC-ABX_3","RC-ABX_5"],
    "RC-ABX_6.0" : ["RC-ABX_5","RC-ABX_7"],
    "RC-ABX_9.0" : ["RC-ABX_8","RC-ABX_11"],
    "RC-ABX_12.5" : ["RC-ABX_11", "RC-ABX_13"]
}

asvnames_asvset = {}
for asvset in asvset_names:
    asvnames_asvset[asvset_names[asvset]] = asvset

sample_interval_exchanges = {}#[Sample interval][metabolite] = Flux
asv_sampleinterval_flux_profiles = {}#[ASV][Sample][Reaction] = Flux
asv_sample_flux_profiles = {}#[ASV][Sample][Reaction] = Flux
rxn_data_hash = {}
asv_biomass = {}
asv_rescaled_biomass = {}
biomasses = {}
for modelid in CommunityFluxSolution:
    sample_interval_exchanges[modelid] = {}
    asv_sample_flux_profiles[modelid] = {}
    base_model = cobra.io.load_json_model("models/"+modelid+'.json')
    for rxnobj in base_model.reactions:
        if rxnobj.id[0:3] == "rxn": 
            (baseid,compartment) = rxnobj.id.split("_")
            if baseid not in rxn_data_hash:
                rxn_data_hash[baseid] = {
                    "id":baseid,
                    "equation":str(rxnobj),
                    "name":rxnobj.name,
                    "obj":rxnobj
                } 
    solution = CommunityFluxSolution[modelid]["solution"]
    biomasses[modelid] = solution["bio1"]
    asv_biomass[modelid] = {}
    asv_rescaled_biomass[modelid] = {}
    for asvname in community_model_data[modelid]["abundances"]:
        asv_biomass[modelid][asvname] = community_model_data[modelid]["abundances"][asvname]*biomasses[modelid]
        asvset = asvnames_asvset[asvname]
        for sample in sample_pairs[modelid]:
            if asvset in sample_asvset_abundances[sample]:
                asv_rescaled_biomass[modelid][asvname] = community_model_data[modelid]["abundances"][asvname]*sample_asvset_abundances[sample][asvset]
            else:
                asv_rescaled_biomass[modelid][asvname] = 0
    for rxn in solution:
        if rxn[0:3] == "EX_":
            sample_interval_exchanges[modelid][rxn[3:11]] = solution[rxn]
        elif rxn[0:3] == "rxn":
            (baseid,compartment) = rxn.split("_")
            if compartment[0:1] == "c":
                index = int(rxn.split("_")[1][1:])
                asvname = community_model_data[modelid]["names"][index-1]
                asvset = asvnames_asvset[asvname]
                if asvname not in asv_sampleinterval_flux_profiles:
                    asv_sampleinterval_flux_profiles[asvname] = {}
                if modelid not in asv_sampleinterval_flux_profiles[asvname]:
                    asv_sampleinterval_flux_profiles[asvname][modelid] = {}
                if asvname not in asv_sample_flux_profiles:
                    asv_sample_flux_profiles[asvname] = {}
                for sample in sample_pairs[modelid]:
                    if sample not in asv_sample_flux_profiles[asvname]:
                        asv_sample_flux_profiles[asvname][sample] = {}
                    if asvset in sample_asvset_abundances[sample]:
                        asv_sample_flux_profiles[asvname][sample][baseid] = solution[rxn]/community_model_data[modelid]["abundances"][asvname]*sample_asvset_abundances[sample][asvset]
                    else:
                        asv_sample_flux_profiles[asvname][sample][baseid] = 0
                asv_sampleinterval_flux_profiles[asvname][modelid][baseid] = solution[rxn]

#Computing metabolite interactions and printing dataframe of scaled ASV flux profiles per sample inteval
record_hash = {}
records = []
asv_metabolite_interactions = {}
for asvname in asv_sampleinterval_flux_profiles:
    asv_metabolite_interactions[asvname] = {}
    for modelid in asv_sampleinterval_flux_profiles[asvname]:
        asv_metabolite_interactions[asvname][modelid] = {}
        for rxn in asv_sampleinterval_flux_profiles[asvname][modelid]:
            if rxn not in record_hash:
                record_hash[rxn] = {
                    "id":rxn,
                    "equation":rxn_data_hash[rxn]["equation"],
                    "name":rxn_data_hash[rxn]["name"]
                }
                records.append(record_hash[rxn])
            for met in rxn_data_hash[rxn]["obj"].metabolites:
                if met.id.split("_")[1] == "e0" and met.id.split("_")[0] in metabolites:
                    if met.id not in asv_metabolite_interactions[asvname][modelid]:
                        asv_metabolite_interactions[asvname][modelid][met.id.split("_")[0]] = 0
                    if rxn_data_hash[rxn]["obj"].metabolites[met] < 0:
                        #This transport is a consumption reaction; consumption should be negative
                        asv_metabolite_interactions[asvname][modelid][met.id.split("_")[0]] += -1*asv_sampleinterval_flux_profiles[asvname][modelid][rxn]
                    else:
                        #This transport is a excretion reaction; excretion should be positive
                        asv_metabolite_interactions[asvname][modelid][met.id.split("_")[0]] += asv_sampleinterval_flux_profiles[asvname][modelid][rxn]       
            record_hash[rxn][asvname+"_"+modelid] = asv_sampleinterval_flux_profiles[asvname][modelid][rxn]/asv_biomass[modelid][asvname] 
df = pd.DataFrame.from_records(records)
df.to_csv(util.output_dir+"/ASVFluxes.csv",index=False)

#Printing ASV metabolite interation matrix
record_hash = {}
records = []
for asvname in asv_metabolite_interactions:
    new_record = {"ASV":asvname}
    for modelid in asv_metabolite_interactions[asvname]:
        for met in asv_metabolite_interactions[asvname][modelid]:
            new_record[met+"_"+modelid] = asv_metabolite_interactions[asvname][modelid][met]
    for met in metabolites:
        total = 0
        count = 0
        for modelid in asv_metabolite_interactions[asvname]:
            if met in asv_metabolite_interactions[asvname][modelid]:
                total += asv_metabolite_interactions[asvname][modelid][met]
                count += 1
        if count > 0:
            new_record[met+"_ave"] = total/count
    records.append(new_record)
df = pd.DataFrame.from_records(records)
df.to_csv(util.output_dir+"/ASVMetaboliteInteractions.csv",index=False)

#Printing dataframe of Sample metabolite interactions per interval and per sample
record_hash = {}
records = [{"Interval":"Name"}]
for met in metabolites:
    records[0][met] = metabolite_names[met]
for modelid in sample_interval_exchanges:
    new_record = {"Interval":modelid}
    for metabolite in sample_interval_exchanges[modelid]:
        new_record[metabolite] = sample_interval_exchanges[modelid][metabolite]
    records.append(new_record)
df = pd.DataFrame.from_records(records)
df.to_csv(util.output_dir+"/SampleIntervalMetaboliteInterations.csv",index=False)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/NewWesternDiet/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


# Printing Sample results from analysis